In [1]:
%load_ext watermark
%watermark

Last updated: 2022-04-04T17:10:20.869941+00:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.22.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 79 Stepping 1, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [2]:
# Librerias
import pandas as pd
import numpy as np

from unicodedata import normalize
import nltk
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model  import LinearRegression as LinReg
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from spacy.lang.es.stop_words import STOP_WORDS
import cloudscraper

import es_core_news_sm
nltk.download('stopwords')
nlp = es_core_news_sm.load()
from datetime import date
from datetime import datetime
from datetime import timedelta

import requests
from bs4 import BeautifulSoup
from parsel import Selector

from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.headless = True

from io import StringIO
import os
import re
import scrapy

import pickle
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import json

from requests.exceptions import ConnectionError
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from scipy.sparse import issparse

# Funciones adicionales
error_page = []
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y")

delta = timedelta(days=1)
ayer = now - delta
ayer = ayer.strftime('%d/%m/%Y')

# Funcion para agregar fechas sin inicio de cero
dt_string_adicional = dt_string[1:]
ayer_adicional = ayer[1:]


def time_date(time):
    fecha = []
    for item in time:
        fecha.append(datetime.fromtimestamp(int(item)).strftime('%d/%m/%Y'))
    return(fecha)

def MesNumero(string):
    m = {'enero': "01",'january': "01",'febrero': "02",'february': "02",'marzo': "03",'march': "03",'abril': "04",'april': "04",'mayo': "05",'may': "05",'junio': "06",'june': "06",'julio': "07",'july': "07",'agosto': "08",'august': "08",'septiembre': "09",'september': "09",'octubre': "10",'october': "10",'noviembre': "11",'november': "11",'diciembre': "12",'december': "12"}
    mcorto = {'ene': "01",'jan': "01",'feb': "02",'mar': "03",'abr': "04",'apr': "04",'may': "05",'jun': "06",'jul': "07",'ago': "08",'aug': "08",'sep': "09",'oct': "10",'nov': "11",'dic': "12",'dec': "12"}

    fecha = string.split("-")
    dia =  fecha[0]
    mes =  fecha[1]
    anio = fecha[2]

    try:
        if len(mes) > 3:
            mes_out = str(m[mes.lower()])
        else:
            mes_out = str(mcorto[mes.lower()])
        
        return(dia + "/" +  mes_out + "/" + anio)
    except:
        raise ValueError('No es un mes')

def limpieza(fuentes):
    agregar_dato=[]
    for item in fuentes:
        agregar_dato.append(item.replace('\xa0', ''))
    return(agregar_dato)

def limpieza_descripcion(variable_corregir):
    valor_limpio = list(map(lambda d: d.replace('\r\n', ''), variable_corregir))
    valor_limpio = [i.strip('\n') for i in valor_limpio]
    valor_limpio = [i.strip('\r') for i in valor_limpio]
    valor_limpio = limpieza(valor_limpio)
    
    variable_almacenamiento = []
    for x in valor_limpio:
        x = remove_tags(x)
        variable_almacenamiento.append(x)
    return (variable_almacenamiento)

def remove_caracter(text):
    text = text.lower()
    text = re.sub('\t ', '', text) 
    return text

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = text.replace(',', '')
    text = text.replace(';', '')
    return TAG_RE.sub('', text)

# COMMAND ----------

def deletestopwords(text):
    word_tokens = nltk.word_tokenize(text)
    stop_words_sp = STOP_WORDS # WITH SPACY
    word_tokens_clean = [each for each in word_tokens if each.lower() not in stop_words_sp and len(each.lower()) > 2]
    return word_tokens_clean

def lemmatizer(text):  
    doc = nlp(text)
    return ' '.join([word.lemma_ for word in doc])

def formatingText(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    text = re.sub('[^a-z ]', '', text)
    return text

def lemant(text1):
    x=text1.lower()
    stopwords= deletestopwords(x)
    stopwords=str(stopwords)
    clean = formatingText(stopwords)
    final = lemmatizer(clean)
    return final

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harold.patino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Machine Learning

In [3]:
with open ("C:/Users/harold.patino/Documents/Periodicos/DeVops/stopwords-es.json") as fname:
    stopwords_es = json.load(fname)
vectorizar_news = TfidfVectorizer(strip_accents='unicode', stop_words=stopwords_es)

In [4]:
# Machine Learning

news = pd.read_csv("C:/Users/harold.patino/Documents/lectura_archivos/Data/Noticias_Data_Entry.csv" , sep=';')
# news = news[['REVIEW','TIP_EVEN','CAT_EVENTO']]

# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

# vectorizador =  TfidfVectorizer(strip_accents='unicode', stop_words=STOP_WORDS)
# vectorizador.fit(news.REVIEW)

# def model_type():
    
#     pipeline_logistica = make_pipeline (
#     vectorizador,
#     DenseTransformer(),
#     LogisticRegression(C=5, verbose=1, random_state=0, penalty='l2')
#     )
#     modelo = pipeline_logistica.fit(X=news.REVIEW, y= news.TIP_EVEN)
#     return modelo


# def model_cat():
   
#     pipeline_logistica = make_pipeline (
#     vectorizador,
#     DenseTransformer(),
#     LogisticRegression(C=5, verbose=1, random_state=0, penalty='l2')
#     )
#     modelo = pipeline_logistica.fit(X=news.REVIEW, y= news.CAT_EVENTO)
#     return modelo

# predCateg = model_cat()
# predType = model_type()

In [ ]:
# modelo_t = pickle.dump(predType, open('model_t.pkl', 'wb'))
# modelo_c = pickle.dump(predCateg, open('model_c.pkl', 'wb'))
# modelo_s = pickle.dump(predSubtype, open('model_s.pkl', 'wb'))

In [5]:
predType = pickle.load(open('model_t.pkl', 'rb'))
predCateg = pickle.load(open('model_c.pkl', 'rb'))
# predSubtype = pickle.load(open('model_s.pkl', 'rb'))

## Periodicos

In [6]:
# El Nuevo Siglo

def fuentes_siglo (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo = sel.css("h3 a::text, span.field.field--name-title.field--type-string.field--label-hidden ::text").extract()
        titulo = [i.replace('\xa0', ' ') for i in titulo]
        categoria = sel.css("h1 ::text").extract()[0]
        descripcion = sel.css("div.body ::text").extract()
        descripcion = limpieza(descripcion)
        descripcion = [i.split('\n\n')[0] for i in descripcion]
        descripcion = [i.split(' \n')[0] for i in descripcion]
        descripcion = pd.DataFrame(descripcion)
        descripcion = descripcion.drop(descripcion[descripcion[0]=='\n     '].index)
        descripcion = descripcion.drop(descripcion[descripcion[0]=='\n       '].index)
        descripcion = descripcion.values.tolist()
        descripcion = [item for lista in descripcion for item in lista]
        descripcion = [i.replace('\n','') for i in descripcion]

        url = sel.css("h3 a::attr(href), h2 a ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4] =='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)
            else:
                if item[:1] =='/':
                    variable = 'https://www.elnuevosiglo.com.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.elnuevosiglo.com.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.clearfix.text-formatted.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item p ::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.strip('\n') for i in descripcion_larga]
                descripcion_larga = [i.strip('\r') for i in descripcion_larga] 
                descripcion_larga = [i.strip('|') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo)== len(descripcion):   
            noticias = {}
            noticias['title'] =  titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  dt_string
            noticias['medium'] = "El Nuevo Siglo"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        else:
            noticias = {}
            noticias['title'] =  titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  dt_string
            noticias['medium'] = "El Nuevo Siglo"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        df_siglo=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_siglo
    
    except ValueError:
        error_page.append(periodico_url)

fuente_nacion = fuentes_siglo('https://www.elnuevosiglo.com.co/seccion/nacion')
fuente_politica = fuentes_siglo('https://www.elnuevosiglo.com.co/seccion/politica')
fuente_economia = fuentes_siglo('https://www.elnuevosiglo.com.co/seccion/economia')

if fuente_nacion is None and fuente_politica is None and fuente_economia is None:
    df_siglo= pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_siglo = pd.concat([fuente_nacion, fuente_politica, fuente_economia])

c:\users\harold.patino\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estara', 'explico', 'expreso', 'habian', 'habra', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'senalo', 'seran', 'seria', 'tendra', 'tendran', 'tenia', 'traves', 'ultima', 'ultimas', 'ultimos'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
c:\users\harold.patino\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estara', 'explico', 'expreso', 'habian', 'habra', 

In [7]:
# El Nuevo Dia

def fuentes_dia (periodico_url):
    try:
        
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)
        if r_noticias.status_code==200:
            
            titulo = sel.css("span.field.field--name-title.field--type-string.field--label-hidden ::text").extract()
            descripcion = sel.css("div.field.field--name-field-entradilla.field--type-string-long.field--label-hidden.field__item ::text").extract()
            descripcion = limpieza_descripcion(descripcion)
            descripcion = pd.DataFrame(descripcion)
            descripcion = descripcion.replace('', 'NaN')
            descripcion = descripcion.drop(descripcion[descripcion[0]=='NaN'].index)
            descripcion = descripcion.values.tolist()
            descripcion = [item for lista in descripcion for item in lista]
            categoria = periodico_url.split('/')[-1]
            fecha = sel.css("time.datetime").extract()
            fecha = [i.split('T')[0] for i in fecha]
            fecha = [i.split('"')[-1] for i in fecha]

            url = sel.css("h2.node__title.title a ::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.elnuevodia.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.elnuevodia.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.clearfix.text-formatted.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item p ::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            fecha_nuevo = []
            for f in fecha:
                f = f.split('-')
                dia = f[2]
                mes = f[1]
                anio = f[0]
                fecha_nuevo.append(dia + "/" +  mes + "/" + anio)

            if len(titulo) == len(descripcion):
                descripcion = descripcion
            else:
                descripcion = titulo

            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0]
                catego.append(predi_categ)
                tipo.append(predi_type)

            if len(titulo) == len(descripcion):          
                noticias = {}
                noticias['title'] =  titulo
                noticias['description'] = descripcion
                noticias['description_long'] = variable_des
                noticias['pubDate'] =  fecha_nuevo
                noticias['medium'] = "El Nuevo Dia"
                noticias['category'] = categoria
                noticias['link'] = variable_url
                #--  Predict  
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego
            else:
                noticias = {}
                noticias['title'] =  titulo
                noticias['description'] = titulo
                noticias['description_long'] = variable_des
                noticias['pubDate'] =  fecha_nuevo
                noticias['medium'] = "El Nuevo Dia"
                noticias['category'] = categoria
                noticias['link'] = variable_url
                #--  Predict  
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego


            df_dia = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category', 'medium', 'processDate', 'predi_type', 'predi_categ'])
            return df_dia
        
        else:
            print(periodico_url) 
    
    except ValueError:
        error_page.append(periodico_url)

fuente_judicial = fuentes_dia('http://www.elnuevodia.com.co/nuevodia/judicial')
fuente_tolima = fuentes_dia('http://www.elnuevodia.com.co/nuevodia/tolima')
fuente_ibague = fuentes_dia('http://www.elnuevodia.com.co/nuevodia/ibague')
fuente_regional = fuentes_dia('http://www.elnuevodia.com.co/nuevodia/archivo-columnista/regional')


if fuente_judicial is None and fuente_tolima is None and fuente_ibague is None and fuente_regional is None:
    df_nuevodia = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_nuevodia = pd.concat([fuente_judicial, fuente_tolima, fuente_ibague, fuente_regional])

In [ ]:
# EL Pais (Cali)
def fuentes_pais (periodico_url):
    try:
        r=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.content, features ="xml")
        sel=Selector(r.text)
        items = soup.findAll('item')

        news_items = []
        for item in items:
            news_item = {}
            news_item['title'] = item.title.text
            news_item['description'] = news_item['title']
            news_item['pubDate'] = item.pubDate.text 
            news_item['pubDate'] = news_item['pubDate'].split(',')[-1]
            news_item['pubDate'] = news_item['pubDate'].split(' ')
            dia = news_item['pubDate'][1]
            mes = news_item['pubDate'][2]
            anio = news_item['pubDate'][3]
            news_item['pubDate'] = (dia + "-" +  mes + "-" + anio)
            news_item['pubDate'] = MesNumero(news_item['pubDate'])
            news_item['category']= item.category.text
            variable_des = ()
            news_item['link'] = item.link.text
            r = requests.get(news_item['link'], headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r.text)
            if r.status_code==200:
                descripcion_larga = sel.css("div.article-content p ::text, div.article-content strong ::text").extract()            
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des = ''.join(valor)
                else:
                    variable_des = ''.join(descripcion_larga)
            else:
                variable_des = ''.join('Error, pagina no encontrada')

            news_item['description_long'] = variable_des
            news_item['medium']="El Pais"  
            #--  Predict
            vect_text1 = lemant(news_item['title'])
            news_item['predi_categ'] = predCateg.predict([vect_text1])[0]
            news_item['predi_type'] = predType.predict([vect_text1])[0] 
            news_items.append(news_item)

        df_noticias_elpais = pd.DataFrame(news_items,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_noticias_elpais
    
    except ValueError:
        error_page.append(periodico_url)

fuente_colombia = fuentes_pais('https://www.elpais.com.co/rss/colombia')
fuente_judicial = fuentes_pais('https://www.elpais.com.co/rss/judicial')
fuente_cali = fuentes_pais('https://www.elpais.com.co/rss/cali')
fuente_valle = fuentes_pais('https://www.elpais.com.co/rss/valle')

if fuente_colombia is None and fuente_judicial is None and fuente_cali is None and fuente_valle is None:
    df_elpais = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_elpais = pd.concat([fuente_colombia, fuente_judicial, fuente_cali, fuente_valle])

In [ ]:
# La Nación y El Lider, dos periodicos misma fuente URL (Huila)
def fuentes_la_nacion(periodico_url):
    try:
        
        r=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.content, features ="xml")
        sel=Selector(r.text)
        items = soup.findAll('item')

        news_items = []
        for item in items:
            news_item = {}
            news_item['title'] = item.title.text
            news_item['description']= item.description.text
            news_item['pubDate'] = item.pubDate.text
            news_item['pubDate'] = news_item['pubDate'].split(',')[-1]
            news_item['pubDate'] = news_item['pubDate'].split(' ')
            dia = news_item['pubDate'][1]
            mes = news_item['pubDate'][2]
            anio = news_item['pubDate'][3]
            news_item['pubDate'] = (dia + "-" +  mes + "-" + anio)
            news_item['pubDate'] = MesNumero(news_item['pubDate'])
            news_item['category']= item.category.text
            variable_des = ()
            news_item['link'] = item.link.text
            r = requests.get(news_item['link'], headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r.text)
            if r.status_code==200:
                descripcion_larga = sel.css("div.penci-entry-content.entry-content p ::text, div.penci-entry-content.entry-content strong ::text").extract()           
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des = ''.join(valor)
                else:
                    variable_des = ''.join(descripcion_larga)
            else:
                variable_des = ''.join('Error, pagina no encontrada')

            news_item['description_long'] = variable_des
            news_item['medium']="La Nacion" 
            #--  Predict
            vect_text1 = lemant(news_item['description'])
            news_item['predi_categ'] = predCateg.predict([vect_text1])[0]
            news_item['predi_type'] = predType.predict([vect_text1])[0] 
            news_items.append(news_item) 
            
        la_nacion = pd.DataFrame(news_items,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return la_nacion
    
    except ValueError:
        error_page.append(periodico_url)

nacion_huila = fuentes_la_nacion('https://www.lanacion.com.co/category/regional-huila/feed/')
nacion_judicial = fuentes_la_nacion('https://www.lanacion.com.co/category/judicial/feed/')

if nacion_huila is None and nacion_judicial is None:
    df_la_nacion = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_la_nacion = pd.concat([nacion_huila, nacion_judicial])

In [ ]:
# Diario del Norte - Nombre Periodico: La Guajira Hoy, Ciudad: Guajira
def fuentes_guajira_hoy(periodico_url):
    try:
        r=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.content, features ="xml")
        sel=Selector(r.text)
        items = soup.findAll('item')


        news_items = []
        for item in items:
            news_item = {}
            news_item['title'] = item.title.text
            news_item['description']= item.description.text
            news_item['description']= remove_tags(news_item['description']).lstrip('<p>')
            news_item['description'] = lemant(news_item['description'])
            news_item['pubDate'] = item.pubDate.text 
            news_item['pubDate'] = news_item['pubDate'].split(',')[-1]
            news_item['pubDate'] = news_item['pubDate'].split(' ')
            dia = news_item['pubDate'][1]
            mes = news_item['pubDate'][2]
            anio = news_item['pubDate'][3]
            news_item['pubDate'] = (dia + "-" +  mes + "-" + anio)
            news_item['pubDate'] = MesNumero(news_item['pubDate'])
            news_item['category']= item.category.text
            variable_des = ()
            news_item['link'] = item.link.text
            r = requests.get(news_item['link'], headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r.text)
            if r.status_code==200:
                descripcion_larga = sel.css("div.td-post-content.tagdiv-type p::text, div.td-post-content.tagdiv-type strong::text").extract()            
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des = ''.join(valor)
                else:
                    variable_des = ''.join(descripcion_larga)
            else:
                variable_des = ''.join('Error, pagina no encontrada')

            news_item['description_long'] = variable_des
            news_item['medium']="La Guajira Hoy"
            #--  Predict
            vect_text1 = news_item['description']
            news_item['predi_categ'] = predCateg.predict([vect_text1])[0]
            news_item['predi_type'] = predType.predict([vect_text1])[0] 
            news_items.append(news_item)


        rss_guajira_hoy = pd.DataFrame(news_items,columns=['title', 'description','description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])   
        return rss_guajira_hoy
    
    except ValueError:
        error_page.append(periodico_url)

guajirahoy_guajira = fuentes_guajira_hoy('https://laguajirahoy.com/seccion/la-guajira/feed')
guajirahoy_riohacha = fuentes_guajira_hoy('https://laguajirahoy.com/seccion/riohacha/feed')
guajirahoy_judicial = fuentes_guajira_hoy('https://laguajirahoy.com/seccion/judiciales/feed')

if guajirahoy_guajira is None and guajirahoy_riohacha is None and guajirahoy_judicial is None:
    df_diarionorte = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_diarionorte = pd.concat([guajirahoy_guajira, guajirahoy_riohacha,guajirahoy_judicial])


In [ ]:
# El Pilon
def fuentes_pilon (periodico_url):
    try:
    
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)  
        if r_noticias.status_code==200:

            titulo = sel.css("h4>a::text").extract()
            descripcion = sel.css('div p::text').extract()
            descripcion_corregida = limpieza_descripcion(descripcion)
            descripcion_corregida = [i.replace('\n',' ') for i in descripcion_corregida]
            fecha = sel.css('li span ::text').extract()
            fecha = [i.replace(', ', ' ') for i in fecha]
            fecha = [i.replace(' ', '-') for i in fecha]
            fecha = [MesNumero(i) for i in fecha]
            categoria = sel.css('section h1::text').extract()
            categoria = list(map(lambda d: d.replace('\n        ', ''), categoria))[0]
            url = sel.css("h4>a ::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://elpilon.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://elpilon.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.land-see-story-copy.max-width-3.mx-auto.py2.px2.ContenidoSingle p ::text, div.land-see-story-copy.max-width-3.mx-auto.py2.px2.ContenidoSingle strong ::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion_corregida:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {} 
            noticias['title'] = titulo
            noticias['description']= descripcion_corregida
            noticias['description_long']= variable_des
            noticias['pubDate'] = fecha
            noticias['category']= categoria
            noticias['link'] = variable_url
            noticias['medium']="El Pilon"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            df_pilon = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_pilon

        else:
            print(periodico_url) 
            
    except ValueError:
        error_page.append(periodico_url)
            
fuente_judicial = fuentes_pilon('https://elpilon.com.co/Noticias/judicial/')
fuente_valledupar = fuentes_pilon('https://elpilon.com.co/Noticias/valledupar/')
fuente_cesar = fuentes_pilon('https://elpilon.com.co/Noticias/cesar-y-la-guajira/cesar/')
fuente_guajira = fuentes_pilon('https://elpilon.com.co/Noticias/cesar-y-la-guajira/la-guajira/')
fuente_municipios = fuentes_pilon('https://elpilon.com.co/Noticias/cesar-y-la-guajira/municipios/')
fuente_sur = fuentes_pilon('https://elpilon.com.co/Noticias/cesar-y-la-guajira/sur/')

if fuente_judicial is None and fuente_valledupar is None and fuente_cesar is None and fuente_guajira is None and fuente_municipios is None and fuente_sur is None:
    df_noticias_pilon = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_noticias_pilon = pd.concat([fuente_judicial, fuente_valledupar, fuente_cesar, fuente_guajira, fuente_municipios, fuente_sur])

In [ ]:
# Diario del Huila
def fuentes_huila (periodico_url):
    try:
        
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text) 

        titulo = sel.css("h2.dg_bm_title a::text").extract()
        titulo = [i.strip('\n') for i in titulo][0:-5]
        titulo = [i.replace('\xa0',' ') for i in titulo]
        descripcion = sel.css("div.post-content p::text").extract()
        descripcion_corregida = limpieza_descripcion(descripcion)[0:-18]
        categoria = sel.css('div.et_pb_text_inner::text').extract()
        categoria = [i.lstrip('Estas viendo la sección de') for i in categoria][0]
        fecha = sel.css('span.published::text').extract()[0:-5]
        fecha_huila = []
        for i in fecha:
            i = i.replace(', ', ' ')
            i = i.replace(' ', '-')
            i = i.split('-')
            dia = i[0]
            mes = i[1]
            anio = i[2]
            fecha_huila.append(MesNumero(dia + "-" + mes + "-" + anio))
        url = sel.css("h2.dg_bm_title a::attr(href)").extract()[0:-5]
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://diariodelhuila.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://diariodelhuila.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.et_pb_module.et_pb_post_content.et_pb_post_content_0_tb_body p::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
    
        if len(titulo) == len(descripcion_corregida):
            descripcion_corregida = descripcion_corregida
        else:
            descripcion_corregida = titulo
        for x in descripcion_corregida:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo) == len(descripcion_corregida):   
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion_corregida
            noticias['description_long'] = variable_des
            noticias['category'] = categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_huila
            noticias['medium']="Diario El Huila"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['category'] = categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_huila
            noticias['medium']="Diario El Huila"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            
        df_huila = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium', 'processDate', 'predi_type', 'predi_categ'])
        return df_huila
    
    except ValueError:
        error_page.append(periodico_url)

fuente_judicial = fuentes_huila("https://diariodelhuila.com/judicial/")
fuente_neiva = fuentes_huila("https://diariodelhuila.com/neiva/")
fuente_regional = fuentes_huila("https://diariodelhuila.com/regional/")

if fuente_judicial is None and fuente_neiva is None and fuente_regional is None:
    df_noticias_huila = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDat': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'], 'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_noticias_huila = pd.concat([fuente_judicial, fuente_neiva, fuente_regional])

In [ ]:
# Prensa Libre
def fuentes_prensalibre (periodico_url):
    try:
        
        r_noticias = requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r_noticias.content, 'html.parser')
        sel=Selector(r_noticias.text) 

        titulo=soup.findAll('h2', class_='title')
        titulo=[i.get_text().strip('\n') for i in titulo]
        descripcion=soup.findAll('div', class_='clrfix')
        descripcion=[i.get_text().strip('\n') for i in descripcion][1:-2]
        descripcion = limpieza(descripcion)
        categoria =soup.findAll('div', class_='pr-post-info')
        categoria =[i.get_text().lower().strip().split(' |')[-1].replace(' ','') for i in categoria]
        enlace =soup.findAll('div', class_='pr-post-info')
        enlace=[i.get_text().lower().strip().split(' |')[-1].split(' /')[-1].replace(' ','') for i in enlace][0]
        enlace = "https://prensalibrecasanare.com/" + enlace
        fecha =soup.findAll('div', class_='pr-post-info')
        fecha =[i.get_text().strip().split(',')[0] for i in fecha]
        fecha_prensa = []
        for i in fecha:
            if i == 'Hoy':
                fecha_prensa.append(dt_string)
            elif i == 'Ayer':
                delta = timedelta(days=1)      
                ayer = now - delta
                ayer = ayer.strftime('%d/%m/%Y')
                fecha_prensa.append(ayer)
            else:
                i = i.split('-') 
                dia = i[0]
                mes = i[1]
                anio = i[2]
                fecha_prensa.append(dia + "/" +  mes + "/" + anio)
                
        url = sel.css("h2 a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://prensalibrecasanare.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://prensalibrecasanare.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.pr-story p::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias ={}    
        noticias['title'] = titulo
        noticias['description']= descripcion
        noticias['description_long']= variable_des
        noticias['category']= categoria
        noticias['link'] =  variable_url
        noticias['pubDate'] = fecha_prensa
        noticias['medium']= 'Prensa Libre Casanare'
        #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_libre = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        df_libre= df_libre.replace('', 'Noticia sin descripción')
        return df_libre
    
    except ValueError:
        error_page.append(periodico_url)


fuente_colombia = fuentes_prensalibre('https://prensalibrecasanare.com/colombia/')
fuente_judicial =fuentes_prensalibre('https://prensalibrecasanare.com/judicial/')
fuente_yopal =fuentes_prensalibre('https://prensalibrecasanare.com/yopal/')
fuente_casanare =fuentes_prensalibre('https://prensalibrecasanare.com/casanare/')

if fuente_colombia is None and fuente_judicial is None and fuente_yopal is None and fuente_casanare is None:
    df_noticias_PrensaLibre = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_noticias_PrensaLibre = pd.concat([fuente_colombia, fuente_judicial, fuente_yopal, fuente_casanare])

In [ ]:
# Extra - Judicial- Nombre Periodico: Mi Curillo Caqueta
def fuentes_extra (periodico_url):

    r_noticias=requests.get(periodico_url, verify=False, headers={"User-Agent": "Mozilla/5.0"})
    sel=Selector(r_noticias.text)
    if r_noticias.status_code==200:
        try:
            titulo = sel.css("h2 >a::text").extract()
            titulo = [i.replace('[VIDEO] ','')for i in titulo]
            categoria = 'Judicial'
            url = sel.css("h2 >a::attr(href)").extract()
            variable_url = []
            variable_des = []
            fecha_extra_m = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://extra.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://extra.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.body-noticia span::text, div.body-noticia p::text, div.body-noticia strong::text, div.body-noticia em::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    fecha_extra =  sel.css("p.date::text").extract()[1]
                    fecha_extra = [fecha_extra]
                    for i in fecha_extra:
                        i = i.split(' /')[0]
                        i = i.replace(' de ','-')
                        i = i.split('-')
                        dia = i[0]
                        mes = i[1]
                        anio = i[2]
                        fecha_extra_m.append(MesNumero(dia + "-" +  mes + "-" + anio))
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            if len(fecha_extra_m) == len(variable_url):
                fecha_extra_m
            else:
                fecha_extra_m = dt_string
            catego = []
            tipo = []
            for x in titulo:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}    
            noticias['title'] = titulo
            noticias['description']= titulo
            noticias['description_long']= variable_des
            noticias['category']= categoria
            noticias['link'] =  variable_url
            noticias['pubDate'] = fecha_extra_m
            noticias['medium']= 'Extra Caqueta'
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            
            extra_judicial = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return extra_judicial

        except ValueError:
            df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

    else:
        df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

    
extra_judicial = fuentes_extra('https://extra.com.co/judicial')
    
if extra_judicial is None:
    df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_extra_judicial = extra_judicial

In [ ]:
# La Tarde - url El Diario
def fuentes_eldiario (periodico_url):
    try:
        
        r_noticias = requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r_noticias.content, 'html.parser')
        sel=Selector(r_noticias.text)

        titulo=soup.findAll('h2', class_='entry-title')
        titulo=[i.get_text() for i in titulo]
        descripcion=soup.findAll('div', class_='entry-summary')
        descripcion=[i.get_text().strip()[0:-153] for i in descripcion]
        fecha=soup.findAll('time', class_='entry-date')
        fecha=[i.get_text() for i in fecha]
        fecha_tarde = []
        for i in fecha:
            i = i.replace(', ', ' ')
            i = i.replace(' ', '-')
            i = i.split('-') 
            dia = i[1]
            mes = i[0]
            anio = i[2]
            fecha_tarde.append(MesNumero(dia + "-" +  mes + "-" + anio))

        categoria=soup.findAll('h1', class_='page-title')
        categoria=[i.get_text().strip().lower().lstrip('local –') for i in categoria][0]
        url = sel.css("h2 a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.eldiario.com.co/categoria/noticias{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.eldiario.com.co/categoria/noticias/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.entry-content span::text, div.entry-content p::text, div.entry-content strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo)== len(descripcion):
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['category'] =categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_tarde
            noticias['medium'] ='El Diario'
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] = descripcion
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['category'] =categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_tarde
            noticias['medium'] ='El Diario'
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego


        df_eldiario=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_eldiario
    
    except ValueError:
        error_page.append(periodico_url)

fuente_pereira =fuentes_eldiario('https://www.eldiario.com.co/categoria/noticias/pereira/')
fuente_risaralda =fuentes_eldiario('https://www.eldiario.com.co/categoria/noticias/risaralda/')
fuente_judicial =fuentes_eldiario('https://www.eldiario.com.co/categoria/judicial/')

if fuente_pereira is None and fuente_risaralda is None and fuente_judicial is None:
    df_latarde = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_latarde = pd.concat([fuente_pereira, fuente_risaralda, fuente_judicial])

In [ ]:
# HSB-Noticias 
def fuentes_hsb(periodico_url):
    try:
        
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)
        if r_noticias.status_code==200:

            titulo = sel.css("h2 a::text").extract()
            titulo = [i.replace('[VIDEO] ','')for i in titulo]
            categoria = sel.css("h2 strong::text").extract()[0]
            url = sel.css("h2 a::attr(href)").extract()
            variable_url = []
            variable_des = []
            fecha_hsb_m = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://hsbnoticias.com{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://hsbnoticias.com/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.body-noticia span::text, div.body-noticia p::text, div.body-noticia strong::text, div.body-noticia em::text").extract() 
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    fecha_hsb =  sel.css("p.date::text").extract()[1]
                    fecha_hsb = [fecha_hsb]
                    for i in fecha_hsb:
                        i = i.split(' /')[0]
                        i = i.replace(' de ','-')
                        i = i.split('-')
                        dia = i[0]
                        mes = i[1]
                        anio = i[2]
                        fecha_hsb_m.append(MesNumero(dia + "-" +  mes + "-" + anio))
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            if len(fecha_hsb_m) == len(variable_url):
                fecha_hsb_m
            else:
                fecha_hsb_m = dt_string        
            catego = []
            tipo = []
            for x in titulo:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = titulo
            noticias['description']= titulo
            noticias['description_long']= variable_des
            noticias['category']= categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_hsb_m
            noticias['medium']="HSB - Noticia"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            hsb = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return hsb
        
        else:
            print(periodico_url)

    except ValueError:
        error_page.append(periodico_url)
    
hsb_nacional = fuentes_hsb('https://hsbnoticias.com/nacional')
hsb_bogota = fuentes_hsb('https://hsbnoticias.com/bogota')
hsb_judicial = fuentes_hsb('https://hsbnoticias.com/judicial')

if hsb_nacional is None and hsb_bogota is None and hsb_judicial is None:
    df_hsb_noticias = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_hsb_noticias = pd.concat([hsb_nacional, hsb_bogota, hsb_judicial])

In [ ]:
# Diario del Sur 
def fuentes_diario_sur(periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        titulo = sel.css("span.field.field--name-title.field--type-string.field--label-hidden::text, div.row div.views-field.views-field-title span.field-content a::text").extract()
        categoria = periodico_url.split('/')[-1]
        dia = sel.css("div.created-day::text").extract()
        mes = sel.css("div.created-month::text").extract()
        anio = sel.css("div.created-year::text").extract()
        anio = [i.replace('/', '') for i in anio]
        fecha = list(zip(dia,mes,anio))
        fecha_sur = []
        for i in fecha:
            dia = i[0]
            mes = i[1]
            anio = i[2]
            fecha_sur.append(MesNumero(dia + "-" +  mes + "-" + anio))
        fecha_1 = [dt_string]
        fecha_sur_1 = fecha_1 + fecha_sur
        url = sel.css("h2.node__title a::attr(href), div.row div.views-field.views-field-title span.field-content a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://diariodelsur.com.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://diariodelsur.com.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.field-item.even p::text, div.field-item.even strong::text, p span::text, div.field__item p::text, div.field__item strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada') 
                
        if len(fecha_sur_1) == len(url):
            fecha_sur_1
        else:
            fecha_sur_1 = dt_string
            
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)
        
       
        noticias = {}
        noticias['title'] = titulo
        noticias['description']= titulo
        noticias['description_long']= variable_des
        noticias['category']= categoria
        noticias['link'] = variable_url
        noticias['pubDate'] = fecha_sur_1
        noticias['medium']="Diario del Sur"
        #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        diariosur = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return diariosur
    
    except ValueError:
        error_page.append(periodico_url)

sur_judicial = fuentes_diario_sur('https://diariodelsur.com.co/judicial')
sur_local = fuentes_diario_sur('https://diariodelsur.com.co/local')
sur_tumaco = fuentes_diario_sur('https://diariodelsur.com.co/tags/tumaco')
sur_ipiales = fuentes_diario_sur('https://diariodelsur.com.co/tags/ipiales')
sur_guachucal = fuentes_diario_sur('https://diariodelsur.com.co/tags/guachucal')
sur_tuquerres = fuentes_diario_sur('http://diariodelsur.com.co/tags/tuquerres')
sur_putumayo = fuentes_diario_sur('https://diariodelsur.com.co/tags/putumayo')

if sur_judicial is None and sur_local is None and sur_tumaco is None and sur_ipiales is None and sur_guachucal is None and sur_tuquerres is None and  sur_putumayo is None:
    df_diario_del_sur = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_diario_del_sur = pd.concat([sur_judicial, sur_local, sur_tumaco, sur_ipiales, sur_guachucal, sur_tuquerres, sur_putumayo])

In [ ]:
# Extra - Judicial- Nombre Periodico: Mi Curillo Caqueta
def fuentes_extra (periodico_url):

    r_noticias=requests.get(periodico_url, verify=False, headers={"User-Agent": "Mozilla/5.0"})
    sel=Selector(r_noticias.text)
    if r_noticias.status_code==200:
        try:
            titulo = sel.css("h2 >a::text").extract()
            titulo = [i.replace('[VIDEO] ','')for i in titulo]
            categoria = 'Judicial'
            url = sel.css("h2 >a::attr(href)").extract()
            variable_url = []
            variable_des = []
            fecha_extra_m = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://extra.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://extra.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.body-noticia span::text, div.body-noticia p::text, div.body-noticia strong::text, div.body-noticia em::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    fecha_extra =  sel.css("p.date::text").extract()[1]
                    fecha_extra = [fecha_extra]
                    for i in fecha_extra:
                        i = i.split(' /')[0]
                        i = i.replace(' de ','-')
                        i = i.split('-')
                        dia = i[0]
                        mes = i[1]
                        anio = i[2]
                        fecha_extra_m.append(MesNumero(dia + "-" +  mes + "-" + anio))
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            if len(fecha_extra_m) == len(variable_url):
                fecha_extra_m
            else:
                fecha_extra_m = dt_string
            catego = []
            tipo = []
            for x in titulo:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}    
            noticias['title'] = titulo
            noticias['description']= titulo
            noticias['description_long']= variable_des
            noticias['category']= categoria
            noticias['link'] =  variable_url
            noticias['pubDate'] = fecha_extra_m
            noticias['medium']= 'Extra Caqueta'
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            
            extra_judicial = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return extra_judicial

        except ValueError:
            df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
            #error_page.append(noticia)

    else:
        df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

        
extra_judicial = fuentes_extra('https://extra.com.co/judicial')
    
if extra_judicial is None:
    df_extra_judicial = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_subtype': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_extra_judicial = extra_judicial

In [ ]:
# Mi Putumayo

def fuentes_putumayo(periodico_url):
    r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
    sel=Selector(r_noticias.text) 
    if r_noticias.status_code==200:
        try:
            descripcion = sel.css("h2.entry-title a::text, h2.entry-title a + br ::text").extract()
            descripcion = [i.strip('br') for i in descripcion]
            fecha = sel.css("time.entry-date ::text").extract()
            fecha_putumayo = []
            for i in fecha:
                    i = i.replace(', ', ' ')
                    i = i.replace(' ', '-')
                    i = i.split('-') 
                    dia = i[0]
                    mes = i[1]
                    anio = i[2]
                    fecha_putumayo.append(MesNumero(dia + "-" +  mes + "-" + anio))

            url = sel.css("h2.entry-title a::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'http://miputumayo.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'http://miputumayo.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.entry-content p::text, div.entry-content strong::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r ',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = descripcion
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] = fecha_putumayo
            noticias['category']="Noticias Generales"
            noticias['link'] = variable_url
            noticias['medium']="Mi Putumayo"
             #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

            putumayo = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return putumayo

        except ValueError:
            df_putumayo = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

    else:
        df_putumayo = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

        
putumayo = fuentes_putumayo('http://miputumayo.com.co/author/MiPutumayo/')
    
if putumayo is None:
    df_putumayo = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_putumayo = putumayo

In [ ]:
# La Cronica del Quindío 
def fuentes_cronica (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        descripcion = sel.css("h2 ::text").extract()
        categoria = sel.css("h1 span ::text").extract()[0].lower()
        url = sel.css("h2 a::attr(href), div.caption a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.cronicadelquindio.com/noticias{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.cronicadelquindio.com/noticias/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200: 
                descripcion_larga = sel.css("div.col-lg-8 p::text, div.col-lg-8 strong::text").extract()
                if descripcion_larga[0] == 'Acceda sin restricciones a todos nuestros contenidos digitales':
                    descripcion_larga = 'Se debe registrarse en la pagina'    
                else:
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = descripcion
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['pubDate'] = dt_string
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium']="La Cronica Del Quindío"
         #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_cronica=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_cronica
    
    except ValueError:
        error_page.append(periodico_url)

fuente_judicial = fuentes_cronica('https://www.cronicadelquindio.com/noticias/judicial')
fuente_region = fuentes_cronica('https://www.cronicadelquindio.com/noticias/region')

if fuente_judicial is None and fuente_region is None:
    df_cronica_quindio = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_cronica_quindio = pd.concat([fuente_judicial, fuente_region])

In [ ]:
# Policia Nacional
def fuentes_policia (periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, verify=False, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            titulo = sel.css("div.views-field.views-field-title a::text, span.views-field.views-field-title a::text").extract()
            descripcion = sel.css("div.views-field.views-field-field-resumen span.field-content ::text").extract()
            descripcion_corregida = limpieza_descripcion(descripcion)
            categoria = 'Noticias Policia'
            fecha = sel.css("div.views-field.views-field-created span.field-content ::text").extract()
            fecha_policia = []
            for i in fecha:
                i = i.split(', ')[-1]
                i = i.replace(' de ','-')
                i = i.split('-') 
                dia = i[0]
                mes = i[1]
                anio = i[2]
                fecha_policia.append(MesNumero(dia + "-" +  mes + "-" + anio))

            url = sel.css("div.views-field.views-field-title a::attr(href), span.views-field.views-field-title a::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.policia.gov.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.policia.gov.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div p::text, div strong::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            if len(titulo) == len(descripcion):
                descripcion_corregida = descripcion
            else:
                descripcion_corregida = titulo           
            catego = []
            tipo = []
            for x in descripcion_corregida:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            if len(titulo) == len(descripcion): 
                noticias = {}
                noticias['title'] = titulo
                noticias['description'] = descripcion_corregida
                noticias['description_long'] = variable_des
                noticias['category'] = categoria 
                noticias['link'] = variable_url
                noticias['pubDate'] = fecha_policia
                noticias['medium']="Policia Nacional"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego
            else:
                noticias = {}
                noticias['title'] = titulo
                noticias['description'] = titulo
                noticias['description_long'] = variable_des
                noticias['category'] = categoria 
                noticias['link'] = variable_url
                noticias['pubDate'] = fecha_policia
                noticias['medium']="Policia Nacional"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego
                
            policia = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return policia

        except requests.exceptions.Timeout:
            print ('Timeout occurred')
        
    except ValueError:
        df_policia = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
        error_page.append(periodico_url)


policia = fuentes_policia('https://www.policia.gov.co/noticias')
    
if policia is None:
    df_policia = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_policia = policia

In [ ]:
# Unidad de Victimas 
def fuentes_unidad(periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, verify=False, timeout=20, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            descripcion = sel.css("h5.tituloh5 a::text").extract()
            categoria = sel.css("h6.ciudadfecha::text").extract()
            categoria = [i.split(' |')[0] for i in categoria]
            fecha = sel.css("h6.ciudadfecha::text").extract()
            fecha_unidad = []
            for i in fecha:
                a = dt_string.split('/')
                i = i.split(', ')[-1]
                i = i.replace(' de ','-')
                i = i.split('-') 
                dia = i[0]
                mes = i[1]
                anio = a[2]
                fecha_unidad.append(MesNumero(dia + "-" +  mes + "-" + anio))
            url = sel.css("h5.tituloh5 a::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, verify=False, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.unidadvictimas.gov.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, verify=False, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.unidadvictimas.gov.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, verify=False, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.row p::text, div.row strong::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = descripcion
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['category'] = categoria
            noticias['link'] = variable_url
            noticias['pubDate'] = fecha_unidad
            noticias['medium'] = 'Unidad de Victimas'
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
            
            unidad_victimas = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return unidad_victimas

        except requests.exceptions.Timeout:
            print ('Timeout occurred')

    except ValueError:
        df_unidadvictimas = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
        error_page.append(periodico_url)

unidadvictimas = fuentes_unidad('https://www.unidadvictimas.gov.co/es/sala-de-prensa/noticias')

if unidadvictimas is None:
    df_unidadvictimas = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_unidadvictimas = unidadvictimas

In [ ]:
# Fuerza Aerea Colombiana 
def fuentes_fuerza(periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, timeout=20, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            url = sel.css("div.post-title a::attr(href)").extract()
            url = pd.DataFrame(url)
            url = url.drop_duplicates([0], keep='first')
            url = url.values.tolist()
            url = [item for lista in url for item in lista]
            l = len(url)
            titulo = sel.css("div.post-title a::text").extract()[:l]
            descripcion = sel.css("div.body p::text").extract()
            descripcion = [e for e in descripcion if e not in ('\xa0', '\n', '\r')]
            variable_url = []
            variable_des = []
            fecha_fuerza = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.fac.mil.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.fac.mil.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("p.MsoNormal span::text, div.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item p::text, div.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item strong::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    fecha = sel.css("div.content-field.field-created::text").extract()
                    for i in fecha:
                        i = i.replace('\n','').replace(' ','').replace('de','-')
                        i = i.split('-')
                        dia = i[0]
                        mes = i[1]
                        anio = i[2]
                        fecha_fuerza.append(MesNumero(dia + "-" +  mes + "-" + anio))
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            if len(fecha_fuerza) == len(variable_url):
                fecha_fuerza
            else:
                fecha_fuerza = dt_string

            if len(titulo) == len(descripcion):
                descripcion = descripcion
            else:
                descripcion = titulo
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            if len(titulo) == len(descripcion):
                noticias = {}
                noticias['title'] = titulo
                noticias['description'] = descripcion
                noticias['description_long'] = variable_des
                noticias['category']= 'Información y prensa'
                noticias['link'] = variable_url
                noticias['medium'] = "Fuerza Aerea Colombiana"
                noticias['pubDate'] = fecha_fuerza

                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego 
            else:
                noticias = {}
                noticias['title'] = titulo
                noticias['description'] = titulo
                noticias['description_long'] = variable_des
                noticias['category']= 'Información y prensa'
                noticias['link'] = variable_url
                noticias['medium'] = "Fuerza Aerea Colombiana"
                noticias['pubDate'] = fecha_fuerza

                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego
            
            fuerza = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return fuerza

        except requests.exceptions.Timeout:
            print ('Timeout ocurred')

    except ValueError:
        df_fuerza =  pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
        error_page.append(noticia)
        
fuerza = fuentes_fuerza('https://www.fac.mil.co/es/noticias')

if fuerza is None:
    df_fuerza  = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_fuerza = fuerza

In [ ]:
# Ejercito
def fuentes_ejercito(periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, timeout=20, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            titulo = sel.css("h5>a::text").extract()
            descripcion = sel.css("p.listaEntradilla ::text").extract()
            descripcion = limpieza_descripcion(descripcion)
            if r_noticias.status_code==200:
                categoria = sel.css("div.titulo-interna::text").extract()[0]
            else:
                categoria = 'Ejercito'
            url = sel.css("h5>a.wrap8::attr(href)").extract()
            fecha = sel.css("p.s_fecha::text").extract()
            variable_url = []
            variable_des = []
            fecha_ejercito = []
            for i in fecha:
                dia = i[0:2]
                dia = dia.replace(' ','')
                i = i.split(' de ')[-1].replace(' ','-').split('-')
                mes = i[0]
                anio= i[1]
                fecha_ejercito.append(MesNumero(dia + "-" +  mes + "-" + anio))
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.ejercito.mil.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.ejercito.mil.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.default_descripcion.row::text").extract()
                    descripcion_larga = limpieza_descripcion(descripcion_larga)
                    descripcion_larga = [i.replace('\n','') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t','') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')   
            catego = []
            tipo = []
            if len(titulo) == len(descripcion):
                descripcion = descripcion
            else:
                descripcion = titulo
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type) 

            if len(titulo) == len(descripcion): 
                noticias = {}
                noticias['title'] = titulo
                noticias['description']= descripcion
                noticias['description_long']= variable_des
                noticias['category']= categoria
                noticias['pubDate'] = fecha_ejercito
                noticias['link'] = variable_url
                noticias['medium']="Ejercito Militar"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego

            else: 
                noticias = {}
                noticias['title'] = titulo
                noticias['description']= titulo
                noticias['description_long']= variable_des
                noticias['category']= categoria
                noticias['pubDate'] = fecha_ejercito
                noticias['link'] = variable_url
                noticias['medium']="Ejercito Militar"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego

            df_militar = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_militar

        except requests.exceptions.Timeout:
            print ('Timeout ocurred')
            
    except ValueError:
        df_ejercito = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
        error_page.append(periodico_url)

ejercito_noticias = fuentes_ejercito("https://www.ejercito.mil.co/informes_noticias/noticias")
ejercito_actualidad = fuentes_ejercito("https://www.ejercito.mil.co/informes_noticias/actualidad")

if ejercito_noticias is None and ejercito_actualidad is None:
    df_ejercito = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_ejercito = pd.concat([ejercito_noticias, ejercito_actualidad])

In [ ]:
# Armada
def fuentes_armada (periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, timeout=20, allow_redirects=False, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            descripcion = sel.css("span>a::text").extract()[:8]
            url = sel.css("span.field-content a::attr(href)").extract()[:8]
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, timeout=20, allow_redirects=False, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.armada.mil.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, timeout=20, allow_redirects=False, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.armada.mil.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, timeout=20, allow_redirects=False, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.field-item.even p::text").extract()
                    descripcion_larga = limpieza_descripcion(descripcion_larga)
                    descripcion_larga = [i.replace('\n','') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t','') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = descripcion
            noticias['description']= descripcion
            noticias['description_long']= variable_des
            noticias['category']= 'Armada'
            noticias['pubDate'] = dt_string
            noticias['link'] = variable_url
            noticias['medium']="Fuerzas Armadas de Colombia"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego 

            armada = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return armada

        except requests.exceptions.Timeout:
            print ('Timeout occurred')
        
    except ValueError:
        df_armada = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
        error_page.append(periodico_url)

armada = fuentes_armada("https://www.armada.mil.co/")

if armada is None:
    df_armada = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_armada = armada

In [ ]:
# El Tiempo

def fuentes_tiempo(periodico_url):
    try:
        r_noticias = requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)
        soup = BeautifulSoup(r_noticias.content, 'html.parser')

        descripcion = soup.findAll('div', class_='epigraph-container')
        descripcion = [i.get_text().strip('\n') for i in descripcion]
        l = len(descripcion)
        titulo = soup.findAll('h3', class_='title-container')
        titulo = [i.get_text().strip('\n') for i in titulo][:l]
        fecha = sel.css("span.published-at::text").extract()[:l]
        fecha_tiempo = []
        for i in fecha:
            variable = i[6:10]
            if variable =='a. m':
                fecha_tiempo.append(dt_string)
            elif variable =='p. m':
                delta = timedelta(days=1)
                ayer = now - delta
                ayer = ayer.strftime('%d/%m/%Y')
                fecha_tiempo.append(ayer)
            elif variable !='a. m' or variable !='p. m':
                i = i.split(' ')
                dia = i[1]
                mes = i[0]
                anio = i[3]
                fecha_tiempo.append(MesNumero(dia + "-" +  mes + "-" + anio))
        url = sel.css("div.epigraph-container a.epigraph.page-link::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.eltiempo.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.eltiempo.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.modulos.public-side p::text, div.modulos.public-side b::text").extract()
                descripcion_larga = [i.replace('\xa0','') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    variable = 'sin descripcion en la noticia'
                    variable_des.append(variable)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        categoria = soup.findAll('h1', class_='oculto')
        categoria = [i.get_text() for i in categoria][0]
        enlace = periodico_url
        medio = "El Tiempo"  
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description']= descripcion
        noticias['description_long'] = variable_des
        noticias['category']= categoria
        noticias['pubDate'] = fecha_tiempo
        noticias['link'] = variable_url
        noticias['medium']= medio
        #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        df = df.replace('', 'Noticia sin descripción')
        df = df.replace('\n', '-',regex=True) 
        return df
    
    except ValueError:
        error_page.append(periodico_url)

def fuentes_tiempo_2 (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        titulo = sel.css('h3>a::text, h2>a::text').extract()
        titulo = [i.replace('\n','')for i in titulo]
        descripcion = sel.css('h3>a::text, div.lead a::text').extract()
        descripcion = limpieza_descripcion(descripcion)
        categoria = sel.css("h1.titulo ::text").extract()[0]
        fecha = sel.css("div.modulo-mas-default meta, div.seccion-fecha meta, div.listing.view-grid meta").extract()
        fecha = [i.split() for i in fecha]
        fecha = pd.DataFrame(fecha)
        fecha = fecha.drop(fecha[fecha[1]!='itemprop="datePublished"'].index)
        fecha = fecha[2]
        fecha = [i.split('=')[-1][1:-2] for i in fecha]
        fecha_tiempo = []
        for i in fecha:
            i = i.split('-') 
            dia = i[2]
            mes = i[1]
            anio = i[0]
            fecha_tiempo.append(dia + "/" +  mes + "/" + anio)
        url = sel.css('h3>a::attr(href), div.lead a::attr(href)').extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.eltiempo.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.eltiempo.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.modulos.public-side p::text, div.modulos.public-side b::text").extract()
                descripcion_larga = [i.replace('\xa0','') for i in descripcion_larga]
                descripcion_larga = [i.replace('⚠️','') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    variable = 'sin descripcion en la noticia'
                    variable_des.append(variable)
                else:
                    variable_des.append(descripcion_larga)         
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0] 
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description']= descripcion
        noticias['description_long']= variable_des
        noticias['category']= categoria
        noticias['link'] = variable_url
        noticias['pubDate'] = fecha_tiempo
        noticias['medium']="El Tiempo"
        #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        df = df.replace('\n', '-',regex=True)
        return df
    
    except ValueError:
        error_page.append(periodico_url)

def fuente_bogota(periodico_url):
    try:
        r_noticias = requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        titulo = sel.css("section.col0 h3.title-container a::text").extract()
        titulo = [i.strip('\n') for i in titulo]
        titulo = pd.DataFrame(titulo)
        titulo = titulo.replace('', 'NaN')
        titulo = titulo.drop(titulo[titulo[0]=='NaN'].index)
        titulo = titulo.values.tolist()
        titulo = [item for lista in titulo for item in lista]
        l = len(titulo)
        descripcion = titulo
        fecha = sel.css("section.col0 div meta").extract()
        fecha = [i.split() for i in fecha]
        fecha = pd.DataFrame(fecha)
        fecha = fecha.drop(fecha[fecha[1]!='itemprop="datePublished"'].index)
        fecha = fecha[2]
        fecha = [i.split('=')[-1][1:-2] for i in fecha][:l]
        fecha_tiempo = []
        for i in fecha:
            i = i.split('-') 
            dia = i[2]
            mes = i[1]
            anio = i[0]
            fecha_tiempo.append(dia + "/" +  mes + "/" + anio) 
        url = sel.css("section.col0 a.title.page-link ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.eltiempo.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.eltiempo.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                    
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.modulos.public-side p::text, div.modulos.public-side b::text, div.lead::text, div.wpb_wrapper p::text, div.wpb_wrapper strong::text, div.wpb_wrapper em::text, div.wpb_wrapper a::text").extract()
                descripcion_larga = [i.replace('\xa0','') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    variable = 'sin descripcion en la noticia'
                    variable_des.append(variable)
                else:
                    variable_des.append(descripcion_larga) 
            else:
                variable_des.append('Error, pagina no encontrada')
        categoria = 'Bogota'
        medio = "El Tiempo"       
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description']= titulo
        noticias['description_long']= variable_des
        noticias['category']= categoria
        noticias['pubDate'] = fecha_tiempo
        noticias['link'] = variable_url
        noticias['medium']= medio
        #--  Predict
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        df = df.replace('\n', '-',regex=True)
        return df
    
    except ValueError:
        error_page.append(periodico_url)

fuente_bogota = fuente_bogota('https://www.eltiempo.com/bogota')
fuente_medellin = fuentes_tiempo('https://www.eltiempo.com/colombia/medellin')
fuente_otras_ciudades = fuentes_tiempo('https://www.eltiempo.com/colombia/otras-ciudades')
fuente_santander = fuentes_tiempo('https://www.eltiempo.com/colombia/santander')
fuente_barranquilla = fuentes_tiempo('https://www.eltiempo.com/colombia/barranquilla')
fuente_cali = fuentes_tiempo('https://www.eltiempo.com/colombia/cali')
fuente_boyaca = fuentes_tiempo_2('https://www.eltiempo.com/noticias/boyaca')
fuente_meta = fuentes_tiempo_2('https://www.eltiempo.com/noticias/meta')

if fuente_bogota is None and fuente_medellin is None and fuente_otras_ciudades is None and fuente_santander is None and fuente_barranquilla is None and fuente_cali is None and fuente_boyaca is None and fuente_meta is None:
    df_noticias_ElTiempo = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_noticias_ElTiempo = pd.concat([fuente_bogota, fuente_medellin, fuente_otras_ciudades, fuente_santander, fuente_barranquilla, fuente_cali, fuente_boyaca, fuente_meta])

In [ ]:
# La voz del Cinaruco
def fuentes_cinaruco (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        titulo = sel.css("h2 a::text ,h3 a ::text").extract()
        descripcion = sel.css('article.col-xs-12.col-sm-4.col-md-4 p, div.well p').extract()
        descripcion = limpieza(descripcion)
        descripcion = [i.replace('<p style="text-align: justify;">','') for i in descripcion]
        descripcion = [i.replace('</strong>','') for i in descripcion]
        descripcion = [i.replace('<strong>','') for i in descripcion]
        descripcion = [i.replace('</p>','') for i in descripcion]
        descripcion = [i.replace('<p>','') for i in descripcion]
        descripcion = pd.DataFrame(descripcion)
        descripcion = descripcion.drop(descripcion[descripcion[0]==''].index)
        descripcion = descripcion.values.tolist()
        descripcion = [item for lista in descripcion for item in lista]
        url = sel.css("h2 a::attr(href) ,h3 a ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://lavozdelcinaruco.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://lavozdelcinaruco.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.panel-body p::text, div.panel-body strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == ' ':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        if len(titulo) == len(descripcion):
            descripcion = descripcion
        else:
            descripcion = titulo

        catego = []
        tipo = []

        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo) == len(descripcion):
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] = dt_string
            noticias['link'] = variable_url
            noticias['category'] = "Noticias"
            noticias['medium'] = "La Voz del Cinaruco"
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['pubDate'] = dt_string
            noticias['link'] = variable_url
            noticias['category'] = "Noticias"
            noticias['medium'] = "La Voz del Cinaruco"
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        cinaruco = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return cinaruco
    
    except ValueError:
        error_page.append(periodico_url)
        
cinaruco = fuentes_cinaruco("https://lavozdelcinaruco.com/")

if cinaruco is None:
    df_cinaruco = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_cinaruco = cinaruco

In [ ]:
# La Patria 

def fuentes_la_patria (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        categoria = sel.css("h1 span ::text").extract()[0]
        descripcion = sel.css("div.content_main.griddiv span.field-content a ::text").extract()
        url = sel.css("div.content_main.griddiv span.field-content a ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.lapatria.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.lapatria.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.field-item.even p::text, div.field-item.even strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == ' ':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = descripcion
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['pubDate'] = dt_string
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium'] = "La Patria"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_la_patria=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_la_patria
    
    except ValueError:
        error_page.append(periodico_url)

fuente_manizales = fuentes_la_patria('https://www.lapatria.com/Manizales')
fuente_caldas = fuentes_la_patria('https://www.lapatria.com/caldas')
fuente_nacional = fuentes_la_patria('https://www.lapatria.com/nacional')

if fuente_manizales is None and fuente_caldas is None and fuente_nacional is None:
    df_la_patria = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_la_patria = pd.concat([fuente_manizales, fuente_caldas, fuente_nacional])

In [ ]:
# El diario del cauca

def fuente_cauca (periodico_url):
    r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
    sel=Selector(r_noticias.text)
    if r_noticias.status_code==200:
        try:
            l = 10
            titulo = sel.css("div.views-field.views-field-title a::text").extract()[:l]
            descripcion = sel.css("div.views-field.views-field-body div.field-content::text").extract()[:l]
            descripcion = limpieza_descripcion(descripcion)
            descripcion = [i.replace('\n\n',' ')  for i in descripcion]
            descripcion = [i.replace('\n',' ')  for i in descripcion]
            categoria = sel.css("h1.page-header::text").extract()[0]
            fecha_cauca = []
            fecha = sel.css("div.created-comment::text").extract()[:l]
            for i in fecha:
                i = i.split(', ')
                dia = i[1]      
                mes = i[0]
                anio = i[2].replace(' ','')
                fecha_cauca.append(MesNumero(dia + "-" +  mes + "-" + anio))

            url = sel.css("div.views-field.views-field-title a::attr(href)").extract()[:l]
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    if item[:1] =='/':
                        variable = 'https://diariodelcauca.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://diariodelcauca.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)

                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.field.field--name-body.field--type-text-with-summary.field--label-hidden.field--item p::text, div.field.field--name-body.field--type-text-with-summary.field--label-hidden.field--item span::text, div.field.field--name-body.field--type-text-with-summary.field--label-hidden.field--item strong::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0]
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] = fecha_cauca
            noticias['category'] = categoria
            noticias['medium'] = "Diario del Cauca"
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

            df_cauca = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_cauca
        
        except ValueError:
            df_cauca = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
    
    else:
        df_cauca = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
    
fuente_judicial = fuente_cauca("https://diariodelcauca.com.co/judicial")
fuente_local = fuente_cauca("https://diariodelcauca.com.co/local")

if fuente_local is None and fuente_judicial is None:
    df_cauca = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_cauca = pd.concat([fuente_judicial, fuente_local])

In [ ]:
# El nuevo liberal
driver = webdriver.Chrome(executable_path='C:/Users/harold.patino/Documents/Periodicos/chromedriver.exe', options=options)

def fuente_nuevo_liberal(periodico_url):
    try:
        driver.get(periodico_url)
        status = [i.text for i in driver.find_elements_by_tag_name('h1')]
        if len(status) > 0:              
            titulo = [i.text for i in driver.find_elements_by_tag_name('h2.jeg_post_title, h3.jeg_post_title')]
            descripcion = [i.text for i in driver.find_elements_by_tag_name('h2.jeg_post_title, div.jeg_post_excerpt p')]
            fecha = [i.text for i in driver.find_elements_by_tag_name('div.jeg_meta_date a')]
            fecha_liberal = []
            for i in fecha:
                if len(i)== 0:
                    i = dt_string
                else:
                    i = i
                fecha_liberal.append(i)
            url = [i.get_attribute('href') for i in driver.find_elements_by_tag_name('h2.jeg_post_title a, div.jeg_post_excerpt a')]
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    driver.get(variable)
                else:
                    if item[:1] =='/':
                        variable = 'https://elnuevoliberal.com{urls}'.format(urls=item)
                        variable_url.append(variable)
                        driver.get(variable)
                    else:
                        variable = 'https://elnuevoliberal.com/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        driver.get(variable)
                if len(titulo)!=0:
                    descripcion_larga = [i.text for i in driver.find_elements_by_tag_name('div.content-inner p, div.content-inner strong, div.content-inner em')]
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {} 
            noticias['title'] = titulo
            noticias['description']= descripcion
            noticias['description_long']= variable_des
            noticias['pubDate'] = fecha_liberal
            noticias['category']= 'Judicial'
            noticias['link'] = variable_url
            noticias['medium']="El Nuevo Liberal"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

            df_nuevo_liberal = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_nuevo_liberal

        else:
            df_nuevo_liberal= pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
    
    except ValueError:
        df_nuevo_liberal= pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})

        
fuente_judicial = fuente_nuevo_liberal('https://elnuevoliberal.com/category/judicial/')

if fuente_judicial is None:
    df_nuevo_liberal= pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_nuevo_liberal = fuente_judicial
    
driver.close()

In [ ]:
# El Heraldo 
def fuentes_el_heraldo (periodico_url):
    try:
        try:
            r_noticias=requests.get(periodico_url, timeout=20, headers={"User-Agent": "Mozilla/5.0"})
            sel=Selector(r_noticias.text)

            descripcion = sel.css("div.text h1 a ::text").extract()
            fecha = sel.css("div.datos ::attr(datetime)").extract()
            fecha_heraldo = []
            for i in fecha:
                i = i.split(' ')[0]
                i = i.split('-') 
                dia = i[2]      
                mes = i[1]
                anio = i[0]
                fecha_heraldo.append(dia + "/" +  mes + "/" + anio)

            url = sel.css("div.text h1 ::attr(href)").extract()
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.elheraldo.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.elheraldo.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("div.field-item.even p::text, div.field-item.even strong::text, div.column.center.content-blocked p::text, div.column.center.content-blocked strong::text, div.column right p::text, div.column right strong::text, p.summary ::text, strong.summary::text").extract()
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0]
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {}
            noticias['title'] = descripcion
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['category'] = [i.split('/')[3] for i in variable_url]
            noticias['pubDate'] = fecha_heraldo
            noticias['medium'] = "El Heraldo"
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

            df_elheraldo=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_elheraldo
        
        except requests.exceptions.RequestException:
            print (periodico_url)  

    except ValueError:
        error_page.append(periodico_url)

fuente_bolivar = fuentes_el_heraldo('https://www.elheraldo.co/bolivar')
fuente_cesar = fuentes_el_heraldo('https://www.elheraldo.co/cesar')
fuente_cordoba = fuentes_el_heraldo('https://www.elheraldo.co/cordoba')
fuente_guajira = fuentes_el_heraldo('https://www.elheraldo.co/la-guajira')
fuente_magdalena = fuentes_el_heraldo('https://www.elheraldo.co/magdalena')
fuente_sucre = fuentes_el_heraldo('https://www.elheraldo.co/sucre')
fuente_san_andres = fuentes_el_heraldo('https://www.elheraldo.co/san-andres')
fuente_local = fuentes_el_heraldo('https://www.elheraldo.co/local')
fuente_judicial = fuentes_el_heraldo('https://www.elheraldo.co/judicial')


if fuente_bolivar is None and fuente_cesar is None and fuente_cordoba is None and fuente_guajira is None and fuente_magdalena is None and fuente_sucre is None and fuente_san_andres is None and fuente_local is None and fuente_judicial is None:
    df_el_heraldo = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_el_heraldo = pd.concat([fuente_bolivar, fuente_cesar, fuente_cordoba, fuente_guajira, fuente_magdalena, fuente_sucre, fuente_san_andres, fuente_local, fuente_judicial])

In [ ]:
# RCN
headers = {"content-Type": "application/x-www-form-urlencoded; charset=UTF-8", "DNT": "1", "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
query_params = {"pageSize": 32, "compSeasons": 274, "altIds": True, "page": 0, "type": "player", "id": -1, "compSeasonId": 274}
cs = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False}, 
                                 delay=10, interpreter='nodejs')

def fuentes_rcn (periodico_url):
    try:
        
        r_noticias = cs.get(periodico_url, headers=headers, params = query_params)
        sel=Selector(r_noticias.text)

        descripcion = sel.css("div.col-sm-12.col-md-12.col-lg-8.border-right h1 a ::text, div.field.field--name-node-title.field--type-ds.field--label-hidden.field__item h1 a::text").extract()
        descripcion = [i.replace('\xa0',' ') for i in descripcion]
        l = len(descripcion)
        categoria = sel.css("div.field.field--name-field-section.field--type-entity-reference.field--label-hidden.field__item a::text, div.field.field--name-field-region.field--type-entity-reference.field--label-hidden.field__item a ::text").extract()[:l]
        fecha = sel.css("div.flex-section div.field.field--name-field-pub-date.field--type-datetime.field--label-hidden.field__item::text, div.second-date div.field.field--name-field-pub-date.field--type-datetime.field--label-hidden.field__item::text").extract()
        fecha_actual = time_date(fecha)
        url = sel.css("div.col-sm-12.col-md-12.col-lg-8.border-right h1 a::attr(href), div.field.field--name-node-title.field--type-ds.field--label-hidden.field__item h1 a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=cs.get(variable, headers=headers, params = query_params)
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com{urls}'.format(urls=item)
                    variable_visual = 'https://www.rcnradio.com{urls}'.format(urls=item)
                    variable_url.append(variable_visual)
                    r_noticias=cs.get(variable, headers=headers, params = query_params)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/{urls}'.format(urls=item)
                    variable_visual = 'https://www.rcnradio.com/{urls}'.format(urls=item)
                    variable_url.append(variable_visual)
                    r_noticias=cs.get(variable, headers=headers, params = query_params)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css('div.item4 p::text, div.item4 strong::text').extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Validar información de la noticia, mediante el link dado en la columna de link')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = descripcion
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['category'] = categoria
        noticias['pubDate'] = fecha_actual
        noticias['medium'] = "RCN Radio"
        noticias['link'] = variable_url
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_rcn = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type', 'predi_categ'])
        return df_rcn
    
    except ValueError:
        error_page.append(periodico_url)

fuente_colombia = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia')
fuente_bogota = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/bogota')
fuente_antioquia = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/antioquia')
fuente_sur = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/sur-del-pais')
fuente_caribe = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/caribe')
fuente_cafetero = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/eje-cafetero')
fuente_llanos = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/llanos')
fuente_pacifico = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/pacifico')
fuente_region_central = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/region-central')
fuente_santanderes = fuentes_rcn('http://webcache.googleusercontent.com/search?q=cache:https://www.rcnradio.com/colombia/santanderes')

if fuente_colombia is None and fuente_bogota is None and fuente_antioquia is None and fuente_sur is None and fuente_caribe is None and fuente_cafetero is None and fuente_llanos is None and fuente_pacifico is None and fuente_region_central is None and fuente_santanderes is None:
    df_rcn_radio = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_rcn_radio = pd.concat([fuente_colombia, fuente_bogota, fuente_antioquia, fuente_sur, fuente_caribe, fuente_cafetero, fuente_llanos, fuente_pacifico, fuente_region_central, fuente_santanderes])

In [ ]:
# Caracol
def fuentes_caracol (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        sel=Selector(r_noticias.text)

        titulo = sel.css("ul.items.layout-automatic-c  h2.module-title a ::text").extract()
        descripcion = sel.css("ul.items.layout-automatic-c p.module-text ::text").extract()
        l = len(titulo)
        categoria = sel.css("h1.headline strong ::text").extract()[0]
        fecha = sel.css("div.module-item-content a::attr(href)").extract()
        fecha = [i.split('/')[-1].split('_')[0]for i in fecha]
        fecha = pd.DataFrame(fecha)
        fecha = fecha.drop(fecha[fecha[0]==''].index)
        fecha = fecha.values.tolist()[:l]
        fecha = [item for lista in fecha for item in lista]
        fecha = time_date(fecha)
        url = sel.css("ul.items.layout-automatic-c  h2.module-title a ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://caracol.com.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://caracol.com.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css('div.cuerpo p::text, div.cuerpo strong::text').extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t','') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        if len(titulo) == len(descripcion):
            descripcion = descripcion
        else:
            descripcion = titulo
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)    

        if len(titulo)== len(descripcion):
            noticias = {}
            noticias['title'] =  titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha
            noticias['medium'] = "Caracol Radio"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] =  titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha
            noticias['medium'] = "Caracol Radio"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        df_caracol=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_caracol
    
    except ValueError:
        error_page.append(periodico_url)

fuente_nacional = fuentes_caracol('https://caracol.com.co/seccion/nacional/')
fuente_regional = fuentes_caracol('https://caracol.com.co/seccion/regional/')
fuente_judicial = fuentes_caracol('https://caracol.com.co/seccion/judicial/')

if fuente_nacional is None and fuente_regional is None and fuente_judicial is None:
    df_caracol = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_caracol = pd.concat([fuente_nacional, fuente_regional, fuente_judicial])

In [ ]:
# Colombiano
def fuentes_colombiano (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo =  sel.css("div.tx-titulo span.priority-content::text, div.noticias-tercer-nivel span.priority-content::text").extract()
        descripcion = sel.css("div.tx-titulo span.priority-content::text, div.noticias-tercer-nivel p:last-child::text").extract()
        descripcion = pd.DataFrame(descripcion)
        descripcion = descripcion.drop(descripcion[descripcion[0]==' '].index)
        descripcion = descripcion.values.tolist()
        descripcion = [item for lista in descripcion for item in lista]
        categoria = sel.css("div.fecha a::text, div.noticias-tercer-nivel div.categoria-noticia a::text").extract()
        fecha = sel.css('div.modulo-principal-seccion div::text, div.noticias-tercer-nivel div::text').extract()
        fecha = [i.split('|')[-1].split('\n')[0].strip(' ') for i in fecha]
        fecha = pd.DataFrame(fecha)
        fecha = fecha.drop(fecha[fecha[0]==''].index)
        fecha = fecha.drop(fecha[fecha[0]=='EXCLUSIVO SUSCRIPTORES'].index)
        fecha = fecha.values.tolist()
        fecha = [item for lista in fecha for item in lista]
        fecha_colombiano = []
        for i in fecha:
            if i[-2:] == 'AM':
                fecha_colombiano.append(dt_string)
            elif i[-2:] == 'PM':
                delta = timedelta(days=1)
                ayer = now - delta
                ayer = ayer.strftime('%d/%m/%Y')
                fecha_colombiano.append(ayer)
            else:
                fecha_colombiano.append(i)
            
        url = sel.css("div.foto a::attr(href),div.left a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.elcolombiano.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.elcolombiano.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.block-text p::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t','') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)     
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
            
        if len(titulo) == len(descripcion): 
            descripcion = descripcion
        else:
            descripcion = titulo
            
        if len(titulo) == len(fecha_colombiano):
            fecha_colombiano
        else:
            fecha_colombiano = dt_string
            
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo) == len(descripcion): 
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha_colombiano
            noticias['medium'] = "Colombiano"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        else:
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha_colombiano
            noticias['medium'] = "Colombiano"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        df_colombiano=pd.DataFrame(noticias,columns=['title', 'description','description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_colombiano
    
    except ValueError:
        error_page.append(periodico_url)

fuente_colombia = fuentes_colombiano('https://www.elcolombiano.com/colombia')
fuente_antioquia = fuentes_colombiano('https://www.elcolombiano.com/antioquia')


if fuente_colombia is None and fuente_antioquia is None:
    df_colombiano = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_colombiano = pd.concat([fuente_colombia, fuente_antioquia])

In [ ]:
# Vanguardia
def fuentes_vanguardia (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        descripcion = sel.css('h1.headline.font-1.bold span::text, div.r01-c01 div.headline.font-1.bold span::text').extract()
        categoria = sel.css("div.header-section a::text").extract()[0]
        url = sel.css("div.template-vg-02 a::attr(href), div.portlet-boundary.portlet-static-end.teaser-viewer-portlet.advanced-vg-54.itr-reloadonpager ::attr(href)").extract()
        url = pd.DataFrame(url)
        url = url.drop(url[url[0]=='https://www.vanguardia.com/exclusivo-suscriptores'].index)
        url = url.drop(url[url[0]=='/judicial'].index)
        url = url.drop(url[url[0]=='/colombia'].index)
        url = url.drop_duplicates(keep='last')
        url = url.values.tolist()
        url =  [item for lista in url for item in lista]
        url_n = []
        for i in url:
            i = i
            numero = i[-5:]
            pagina = i[0:4]
            if numero.isnumeric() or pagina =='http':
                url_n.append(i)
        variable_url = []
        variable_des = []
        for item in url_n:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.vanguardia.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.vanguardia.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.paragraph p::text, div.paragraph span::text, div.paragraph b::text, div.paragraph a::text, div.paragraph strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] =  descripcion
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['pubDate'] =  dt_string
        noticias['medium'] = "Vanguardia"
        noticias['category'] = categoria
        noticias['link'] = variable_url
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_vanguardia = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_vanguardia
    
    except ValueError:
        error_page.append(periodico_url)

def fuentes_vanguardia_2 (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo = sel.css("div.r01-c01 div.headline.font-1.bold span::text").extract()
        descripcion = sel.css("div.r01-c01 p:first-child::text").extract()
        descripcion = pd.DataFrame(descripcion)
        descripcion = descripcion.drop(descripcion[descripcion[0]==' '].index)
        descripcion = descripcion.values.tolist()
        descripcion = [item for lista in descripcion for item in lista]
        l = len(descripcion)
        categoria = sel.css("div.header-section a::text").extract()[0]
        fecha = sel.css("div.top div.date::text").extract()
        fecha = [i.split('\n')[0].strip(' ').split(' ,')[0].replace(' ','-')for i in fecha]
        fecha_v = []
        for i in fecha:
            if i[-5:] == 'horas' or i[0:4]=='hace':
                fecha_v.append(dt_string)
            else:
                fecha_v.append(MesNumero(i))

        url = sel.css("div.template-vg-28 a::attr(href)").extract()
        url = pd.DataFrame(url)
        url = url.drop(url[url[0]=='https://www.vanguardia.com/exclusivo-suscriptores'].index)
        url = url.drop_duplicates(keep='last')
        url = url.values.tolist()
        url =  [item for lista in url for item in lista]
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.vanguardia.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.vanguardia.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=False)
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.paragraph p::text, div.paragraph span::text, div.paragraph b::text, div.paragraph a::text, div.paragraph strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo)== len(descripcion):
            noticias = {}
            noticias['title'] =  titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha_v[:l]
            noticias['medium'] = "Vanguardia"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] =  descripcion
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] =  fecha_v[:l]
            noticias['medium'] = "Vanguardia"
            noticias['category'] = categoria
            noticias['link'] = variable_url
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        df_vanguardia=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_vanguardia
    
    except ValueError:
        error_page.append(periodico_url)

fuente_judicial = fuentes_vanguardia('https://www.vanguardia.com/judicial')
fuente_colombia = fuentes_vanguardia('https://www.vanguardia.com/colombia')
fuente_bucaramanga = fuentes_vanguardia_2('https://www.vanguardia.com/area-metropolitana/bucaramanga')
fuente_floridablanca = fuentes_vanguardia_2('https://www.vanguardia.com/area-metropolitana/floridablanca')
fuente_giron = fuentes_vanguardia_2('https://www.vanguardia.com/area-metropolitana/giron')
fuente_piedecuesta = fuentes_vanguardia_2('https://www.vanguardia.com/area-metropolitana/piedecuesta')
fuente_barrancabermeja = fuentes_vanguardia_2('https://www.vanguardia.com/santander/barrancabermeja')
fuente_guanenta = fuentes_vanguardia_2('https://www.vanguardia.com/santander/guanenta')
fuente_comunera = fuentes_vanguardia_2('https://www.vanguardia.com/santander/comunera')
fuente_region = fuentes_vanguardia_2('https://www.vanguardia.com/santander/region')

if fuente_judicial is None and fuente_colombia is None and fuente_bucaramanga is None and fuente_floridablanca is None and fuente_giron is None and fuente_piedecuesta is None and fuente_barrancabermeja is None and fuente_guanenta is None and fuente_comunera is None and fuente_region is None:
    df_vanguardia = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_vanguardia = pd.concat([fuente_judicial, fuente_colombia, fuente_bucaramanga, fuente_floridablanca, fuente_giron, fuente_piedecuesta, fuente_barrancabermeja, fuente_guanenta, fuente_comunera, fuente_region])

In [ ]:
# Blu Radio
def fuentes_blu (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo = sel.css("h3.PromoC-title a::text, h3.PromoB-title div.PromoB-title-touch a::text").extract()
        titulo = limpieza_descripcion(titulo)
        titulo =  [i.split('\n  ')[0]for i in titulo]
        categoria = sel.css("h1.SectionPage-pageHeading::text").extract()[0]
        descripcion = sel.css("h3.PromoC-title a::text, div.PromoB-description a::text").extract()
        descripcion = limpieza_descripcion(descripcion)
        descripcion = [i.split('\n  ')[0]for i in descripcion]
        fecha = sel.css("div.PromoC-timestamp::attr(data-timestamp), div.PromoB-timestamp::attr(data-timestamp)").extract()
        fecha_b = [int(i)//1000 for i in fecha]
        fecha_b = time_date(fecha_b)
        url = sel.css("h3.PromoC-title a::attr(href), div.PromoB-description a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.bluradio.com/blu360{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.bluradio.com/blu360/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.Quote blockquote::text,div.RichTextArticleBody-body.RichTextBody p::text, div.RichTextArticleBody-body.RichTextBody b::text, div.RichTextArticleBody-body.RichTextBody strong::text, div.RichTextArticleBody-body.RichTextBody a::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r','') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] =  titulo
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['pubDate'] =  fecha_b
        noticias['medium'] = "Blu Radio"
        noticias['category'] = categoria
        noticias['link'] = variable_url
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_blu=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_blu
    
    except ValueError:
        error_page.append(periodico_url)

fuente_360 = fuentes_blu('https://www.bluradio.com/blu360')
fuente_nacion = fuentes_blu('https://www.bluradio.com/nacion')
fuente_bogota = fuentes_blu('https://www.bluradio.com/blu360/bogota')
fuente_antioquia = fuentes_blu('https://www.bluradio.com/blu360/antioquia')
fuente_pacifico = fuentes_blu('https://www.bluradio.com/blu360/pacifico')
fuente_caribe = fuentes_blu('https://www.bluradio.com/blu360/caribe')
fuente_santanderes = fuentes_blu('https://www.bluradio.com/blu360/santanderes')


if fuente_360 is None and fuente_nacion is None and fuente_bogota is None and fuente_antioquia is None and fuente_pacifico is None and fuente_caribe is None and fuente_santanderes is None:
    df_blu = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_blu = pd.concat([fuente_360, fuente_nacion, fuente_bogota, fuente_antioquia, fuente_pacifico ,fuente_caribe, fuente_santanderes])

In [ ]:
# El Universal
def fuentes_universal (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        categoria = sel.css("p.section-name ::text").extract()[0]
        descripcion = sel.css("div.general div.headline span.priority-content ::text").extract()
        l = len(descripcion)
        fecha = sel.css("div.general div.fecha ::text").extract()
        fecha_universal = []
        for i in fecha:
            a = dt_string.split('/')    
            i= i.split(' ')
            if i[0] == '':
                dia = i[2]
                mes = i[1]
            else:
                dia = i[1]
                mes = i[0]
            anio = a[2]
            fecha_universal.append(MesNumero(dia + "-" +  mes + "-" + anio))

        url = sel.css("div.info a::attr(href)").extract()[:l]
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.eluniversal.com.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.eluniversal.com.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.paragraph p::text, div.paragraph strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = descripcion
        noticias['description'] = descripcion
        noticias['description_long'] = variable_des
        noticias['pubDate'] = fecha_universal
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium'] = "El Universal"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        df_el_universal = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_el_universal
    
    except ValueError:
        error_page.append(periodico_url)

fuente_cartagena = fuentes_universal("https://www.eluniversal.com.co/cartagena")
fuente_regional = fuentes_universal("https://www.eluniversal.com.co/regional")
fuente_colombia = fuentes_universal("https://www.eluniversal.com.co/colombia")

if fuente_cartagena is None and fuente_regional is None and fuente_colombia is None:
    df_universal = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_universal = pd.concat([fuente_cartagena, fuente_regional, fuente_colombia])

In [ ]:
# Hoy Diario del Magdalena
def fuentes_magdalena (periodico_url):
    try:  
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo1 = sel.css("div.col-sm-12 h2.title a::text").extract()
        titulo1 = [i.replace('\n', '') for i in titulo1]
        titulo2 = sel.css("div.listing.listing-blog.listing-blog-5.clearfix h2.title a::text").extract()
        titulo2 = [i.replace('\n', '') for i in titulo2]
        titulo = titulo1 + titulo2
        descripcion1 = sel.css("div.col-sm-12 h2.title a::text").extract()
        descripcion1 = [i.replace('\n', '') for i in descripcion1]
        descripcion2 = sel.css("div.listing.listing-blog.listing-blog-5.clearfix div.post-summary ::text").extract()
        descripcion2 = [i.replace('\xa0', ' ') for i in descripcion2]
        descripcion2 = limpieza_descripcion(descripcion2)
        descripcion2 = [i.replace('\n','') for i in descripcion2]
        descripcion2 = pd.DataFrame(descripcion2)
        descripcion2 = descripcion2.drop(descripcion2[descripcion2[0]==' '].index)
        descripcion2 = descripcion2.drop(descripcion2[descripcion2[0]==''].index)
        descripcion2 = descripcion2.drop(descripcion2[descripcion2[0]=='lee mas...'].index)
        descripcion2 = descripcion2.values.tolist()
        descripcion2 = [item for lista in descripcion2 for item in lista]
        descripcion = descripcion1 + descripcion2
        fecha1 = [dt_string] * len(titulo1)
        fecha2 = sel.css("div.listing.listing-blog.listing-blog-5.clearfix span.time ::text").extract()
        fecha_magdalena2 = []
        for i in fecha2:
            i = i.split(' ')[0].split('/')
            dia = i[0]      
            mes = i[1]
            anio = i[2]
            fecha_magdalena2.append(dia + "/" +  mes + "/" + anio)
        fecha = fecha1 + fecha_magdalena2
        medio = "Hoy Diario de Magdalena"
        categoria = sel.css("h1.page-heading ::text").extract()[0]
        url = sel.css("div.col-sm-12 h2.title a::attr(href), div.listing.listing-blog.listing-blog-5.clearfix h2.title a::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.hoydiariodelmagdalena.com.co/archivos{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.hoydiariodelmagdalena.com.co/archivos/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.entry-content.clearfix.single-post-content p::text, div.entry-content.clearfix.single-post-content strong::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')

        catego = []
        tipo = []
        
        if len(titulo) == len(descripcion): 
            descripcion = descripcion
        else:
            descripcion = titulo
        for x in descripcion:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        if len(titulo) == len(descripcion):
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = descripcion
            noticias['description_long'] = variable_des
            noticias['pubDate'] = fecha
            noticias['category'] = categoria
            noticias['link'] = variable_url
            noticias['medium'] = medio
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego
        else:
            noticias = {}
            noticias['title'] = titulo
            noticias['description'] = titulo
            noticias['description_long'] = variable_des
            noticias['pubDate'] = fecha
            noticias['category'] = categoria
            noticias['link'] = variable_url
            noticias['medium'] = medio
            #--  Predict  
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

        df_magdalena = pd.DataFrame(noticias, columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return df_magdalena
    
    except ValueError:
        error_page.append(periodico_url)
    

fuente_magdalena = fuentes_magdalena("https://www.hoydiariodelmagdalena.com.co/archivos/category/magdalena")
fuente_locales = fuentes_magdalena("https://www.hoydiariodelmagdalena.com.co/archivos/category/locales")
fuente_regional = fuentes_magdalena("https://www.hoydiariodelmagdalena.com.co/archivos/category/regional")
fuente_judicial = fuentes_magdalena("https://www.hoydiariodelmagdalena.com.co/archivos/category/judicial")
fuente_judicialN = fuentes_magdalena("https://www.hoydiariodelmagdalena.com.co/archivos/category/judicial-nacional")


if fuente_magdalena is None and fuente_locales is None and fuente_regional is None and fuente_judicial is None and fuente_judicialN is None:
    df_magdalena = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_magdalena = pd.concat([fuente_magdalena, fuente_locales, fuente_regional, fuente_judicial, fuente_judicialN])

In [ ]:
# La Opinión
def fuentes_la_opinion (periodico_url):
    try:
        r_noticias=requests.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})
        r_noticias.encoding = 'utf-8'
        sel=Selector(r_noticias.text)

        titulo = sel.css("section.section span.field-content a::text").extract()
        url = sel.css("section.section span.field-content a::attr(href)").extract()
        categoria = sel.css("h1.title ::text").extract()[0]
        variable_url = []
        variable_des = []
        fecha_opinion = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.laopinion.com.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.laopinion.com.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.clearfix.text-formatted.field.field--name-field-free-text.field--type-text-long.field--label-hidden.field__item p::text").extract()
                descripcion_larga = limpieza_descripcion(descripcion_larga)
                descripcion_larga = ' '.join(descripcion_larga)
                fecha = sel.css("span.field.field--name-created.field--type-created.field--label-hidden::text").extract()
                for i in fecha:
                    i = i.split(',')[-1]
                    i = i.replace(' de ','-').replace(' ','')
                    i = i.split('-')
                    dia = i[0]
                    mes = i[1]
                    anio = i[2]
                    fecha_opinion.append(MesNumero(dia + "-" +  mes + "-" + anio))
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
                
        if len(fecha_opinion) == len(variable_url):
            fecha_opinion
        else:
            fecha_opinion = dt_string
        
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description'] = titulo
        noticias['description_long'] = variable_des
        noticias['pubDate'] = fecha_opinion
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium']="La Opinion"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        opinion=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return opinion
    
    except ValueError:
        error_page.append(periodico_url)

fuente_cucuta = fuentes_la_opinion('https://www.laopinion.com.co/cucuta')
fuente_catatumbo = fuentes_la_opinion('https://www.laopinion.com.co/region/catatumbo')
fuente_pamplona = fuentes_la_opinion('https://www.laopinion.com.co/region/pamplona')
fuente_judicial = fuentes_la_opinion('https://www.laopinion.com.co/judicial')
fuente_ocaña = fuentes_la_opinion('https://www.laopinion.com.co/region/ocana')

if fuente_cucuta is None and fuente_catatumbo is None and fuente_pamplona is None and fuente_judicial is None and fuente_ocaña is None:
    df_opinion = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_opinion = pd.concat([fuente_cucuta, fuente_catatumbo, fuente_ocaña, fuente_pamplona, fuente_judicial])

In [ ]:
# La Razón
def fuentes_la_razon (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        sel=Selector(r_noticias.text)

        titulo = sel.css("h2 a::text, div.meta.default-meta.side a::text, h4.bold.xt-post-title a::text").extract()
        url = sel.css("h2 a::attr(href), div.meta.default-meta.side a::attr(href), h4.bold.xt-post-title a::attr(href)").extract()
        categoria = periodico_url.split('/')[3]
        variable_url = []
        variable_des = []
        fecha_razon = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://larazon.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://larazon.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.post-body p::text, div.post-body strong::text, div.post-body br::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                fecha = sel.css("time.xt-post-date ::attr(datetime)").extract()[0]
                fecha = fecha.split('T')[0].split('-') 
                fecha_razon.append(fecha[2] + "/" +  fecha[1] + "/" + fecha[0])
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
                
        if len(fecha_razon) == len(variable_url):
            fecha_razon
        else:
            fecha_razon = dt_string
            
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description'] = titulo
        noticias['description_long'] = variable_des
        noticias['pubDate'] = fecha_razon
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium'] = "La Razon"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        la_razon = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return la_razon
    
    except ValueError:
        error_page.append(periodico_url)

def fuentes_la_razon_1 (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        sel=Selector(r_noticias.text)

        titulo = sel.css("h3.xt-archive-post-title.xt-post-title a::text").extract()
        fecha = sel.css("time.xt-post-date ::text").extract()
        fecha_razon = []
        for i in fecha:
            i = i.replace(',','')
            i = i.split(' - ')[0].split(' ')
            dia = i[0]      
            mes = i[1]
            anio = i[2]
            fecha_razon.append(MesNumero(dia + "-" +  mes + "-" + anio))
        url = sel.css("h3.xt-archive-post-title.xt-post-title a::attr(href)").extract()
        categoria = [i.split('/')[3] for i in url][0]
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://larazon.co{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://larazon.co/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("div.post-body p::text, div.post-body strong::text, div.post-body br::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == ' ':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description'] = titulo
        noticias['description_long'] = variable_des
        noticias['pubDate'] = fecha_razon
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium'] = "La Razon"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        la_razon = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return la_razon
    
    except ValueError:
        error_page.append(periodico_url)

fuente_cordoba = fuentes_la_razon('https://larazon.co/seccion-cordoba/')
fuente_monteria = fuentes_la_razon('https://larazon.co/seccion-monteria/')
fuente_judicial = fuentes_la_razon('https://larazon.co/judiciales/')
fuente_nacion = fuentes_la_razon_1('https://larazon.co/nacion/')


if fuente_cordoba is None and fuente_monteria is None and fuente_judicial is None and fuente_nacion is None:
    df_la_razon = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_la_razon = pd.concat([fuente_cordoba, fuente_monteria, fuente_judicial, fuente_nacion])

In [ ]:
# El Espectador (Colombia)
def fuentes_el_espectador (periodico_url):
    try:
        r_noticias=requests.get(periodico_url)
        sel=Selector(r_noticias.text)

        titulo = sel.css("h2.Card-Title.Title.Title a ::text").extract()
        fecha = sel.css("p.Card-Datetime.Datetime ::text").extract()
        categoria = periodico_url.split('/')[-2]
        fecha_espectador = []
        for i in fecha:
            if i[-5:] == 'horas' or i[0:4]=='Hace':
                fecha_espectador.append(dt_string)
            else:
                i = i.split('  - ')[0].split(' ')
                dia = i[0]      
                mes = i[1]
                anio = i[2]
                fecha_espectador.append(MesNumero(dia + "-" +  mes + "-" + anio))

        url = sel.css("h2.Card-Title.Title.Title a ::attr(href)").extract()
        variable_url = []
        variable_des = []
        for item in url:
            if item[0:4]=='http':
                variable = item
                variable_url.append(item)
                r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                r_noticias.encoding = 'utf-8'
                sel=Selector(r_noticias.text)  
            else:
                if item[:1] =='/':
                    variable = 'https://www.elespectador.com{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
                else:
                    variable = 'https://www.elespectador.com/{urls}'.format(urls=item)
                    variable_url.append(variable)
                    r_noticias=requests.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)
            if r_noticias.status_code==200:
                descripcion_larga = sel.css("p.font--secondary::text,div.Hook.ArticleHeader-Hook.Hook_main div::text,div.Hook.ArticleHeader-Hook.Hook_main::text").extract()
                descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                descripcion_larga = [i.replace('\r',' ') for i in descripcion_larga]
                descripcion_larga = ' '.join(descripcion_larga)
                if descripcion_larga == '':
                    valor = 'Sin descripcion en la noticia'
                    variable_des.append(valor)
                else:
                    variable_des.append(descripcion_larga)
            else:
                variable_des.append('Error, pagina no encontrada')
        catego = []
        tipo = []
        for x in titulo:
            vect_text1 = lemant(x)
            predi_categ = predCateg.predict([vect_text1])[0]
            predi_type = predType.predict([vect_text1])[0]
            catego.append(predi_categ)
            tipo.append(predi_type)

        noticias = {}
        noticias['title'] = titulo
        noticias['description'] = titulo
        noticias['description_long'] = variable_des
        noticias['pubDate'] = fecha_espectador
        noticias['category'] = categoria
        noticias['link'] = variable_url
        noticias['medium'] = "El Espectador"
        #--  Predict  
        noticias['predi_type'] = tipo
        noticias['predi_categ'] = catego

        el_espectador = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
        return el_espectador
    
    except ValueError:
        error_page.append(periodico_url)

fuente_medellin = fuentes_el_espectador('https://www.elespectador.com/colombia/medellin/')
fuente_cali = fuentes_el_espectador('https://www.elespectador.com/colombia/cali/')
fuente_barranquilla = fuentes_el_espectador('https://www.elespectador.com/colombia/barranquilla/')
fuente_cartagena = fuentes_el_espectador('https://www.elespectador.com/colombia/cartagena/')
fuente_cundinamarca = fuentes_el_espectador('https://www.elespectador.com/colombia/cundinamarca/')

if fuente_medellin is None and fuente_cali is None and fuente_barranquilla is None and fuente_cartagena is None and fuente_cundinamarca is None:
    df_el_espectador = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_el_espectador = pd.concat([fuente_medellin, fuente_cali, fuente_barranquilla,fuente_cartagena, fuente_cundinamarca])

In [ ]:
# El Informador
cs = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False}, 
                                 delay=5, interpreter='nodejs')

def fuentes_el_informador (periodico_url):
    try:
       
        r_noticias = cs.get(periodico_url, headers={"User-Agent": "Mozilla/5.0"})

        sel=Selector(r_noticias.text)
        soup = BeautifulSoup(r_noticias.content, 'html.parser')
        if r_noticias.status_code==200:
            titulo=soup.findAll('h2', class_='article-title')
            titulo=[i.get_text().strip('\n\t') for i in titulo]
            descripcion = soup.findAll('section', class_='article-intro')
            descripcion=[i.get_text().replace('\t','') for i in descripcion]
            descripcion=[i.replace('\n','') for i in descripcion]
            descripcion=[i.strip('\xa0') for i in descripcion]
            categoria=soup.findAll('small', class_='subheading-category')
            categoria=[i.get_text().strip() for i in categoria][0]
            fecha =  sel.css("dd.published.hasTooltip meta").extract() 
            fecha = [i.split() for i in fecha]
            fecha = pd.DataFrame(fecha)
            fecha = fecha.drop(fecha[fecha[1]!='itemprop="datePublished"'].index)
            fecha = fecha[2]
            fecha = [i.split('=')[-1][1:-2].split('T')[0] for i in fecha]
            fecha_informador = []
            for i in fecha:
                i = i.split('-') 
                dia = i[2]
                mes = i[1]
                anio = i[0]
                fecha_informador.append(dia + "/" +  mes + "/" + anio)
            if len(fecha_informador) == len(titulo):
                fecha_informador = fecha_informador
            else:
                fecha_informador = dt_string
            url =  sel.css("h2.article-title a::attr(href)").extract() 
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    r_noticias=cs.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                    r_noticias.encoding = 'utf-8'
                    sel=Selector(r_noticias.text)  
                else:
                    if item[:1] =='/':
                        variable = 'https://www.elinformador.com.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=cs.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                    else:
                        variable = 'https://www.elinformador.com.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        r_noticias=cs.get(variable, headers={"User-Agent": "Mozilla/5.0"})
                        r_noticias.encoding = 'utf-8'
                        sel=Selector(r_noticias.text)
                if r_noticias.status_code==200:
                    descripcion_larga = sel.css("section.article-content p::text, section.article-content br::text, section.article-content::text").extract()
                    descripcion_larga = [i.replace('\r', '') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t', '') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\xa0', '') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n', '') for i in descripcion_larga]
                    descripcion_larga =  ' '.join(descripcion_larga)
                    if descripcion_larga == ' ':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')

            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            if len(titulo)== len(descripcion):
                noticias = {}
                noticias['title'] = titulo
                noticias['description']= descripcion
                noticias['description_long']= variable_des
                noticias['category']= categoria
                noticias['link'] = variable_url
                noticias['pubDate'] = fecha_informador
                noticias['medium']="El Informador"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego   
            else:
                noticias = {}
                noticias = {}
                noticias['title'] = titulo
                noticias['description']= titulo
                noticias['description_long']= variable_des
                noticias['category']= categoria
                noticias['link'] = variable_url
                noticias['pubDate'] = fecha_informador
                noticias['medium']="El Informador"
                #--  Predict
                noticias['predi_type'] = tipo
                noticias['predi_categ'] = catego 

            df_el_informador=pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_el_informador
        
        else:
            print(periodico_url)


    except requests.exceptions.RequestException:
        print (periodico_url) 

fuente_bolivar = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/181-bolivar')
fuente_distrito = fuentes_el_informador('https://www.elinformador.com.co/index.php/el-magdalena/81-distrito')
fuente_cienaga = fuentes_el_informador('https://www.elinformador.com.co/index.php/el-magdalena/82-cienaga')
fuente_departamento = fuentes_el_informador('https://www.elinformador.com.co/index.php/el-magdalena/83-departamento')
fuente_zona = fuentes_el_informador('https://www.elinformador.com.co/index.php/el-magdalena/219-zona-critica')
fuente_guajira = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/77-la-guajira')
fuente_atlantico = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/180-atlantico')
fuente_cesar = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/182-cesar')
fuente_cordoba = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/183-cordoba')
fuente_sucre = fuentes_el_informador('https://www.elinformador.com.co/index.php/region-caribe/184-sucre')
fuente_judiciales = fuentes_el_informador('https://www.elinformador.com.co/index.php/judiciales/71-judiciales-local')
fuente_judicialesN = fuentes_el_informador('https://www.elinformador.com.co/index.php/judiciales/72-judiciales-nacional')

if fuente_bolivar is None and fuente_distrito is None and fuente_cienaga is None and fuente_departamento is None  and fuente_zona is None  and fuente_guajira is None  and fuente_atlantico is None  and fuente_cesar is None  and fuente_cordoba is None  and fuente_sucre is None  and fuente_judiciales is None  and fuente_judicialesN is None:
    df_el_informador = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_el_informador = pd.concat([fuente_bolivar, fuente_distrito, fuente_cienaga , fuente_departamento, fuente_zona, fuente_guajira, fuente_atlantico, fuente_cesar, fuente_cordoba, fuente_sucre, fuente_judiciales, fuente_judicialesN])

In [ ]:
# El meridiano

driver = webdriver.Chrome(executable_path='C:/Users/harold.patino/Documents/Periodicos/chromedriver.exe', options=options)
def fuente_meridiano(periodico_url):
    try:
        driver.implicitly_wait(10)
        driver.get(periodico_url)
        status = [i.text for i in driver.find_elements_by_tag_name('span.pl-0.border__span')]
        if len(status) != 0:              
            titulo = [i.text for i in driver.find_elements_by_tag_name('h2.News-content-title')]
            l = len(titulo)
            descripcion = [i.text for i in driver.find_elements_by_tag_name('p.News-content-description')]
            descripcion = [i for i in descripcion if i !='']
            categoria = [i.text for i in driver.find_elements_by_tag_name('span.News-content-details-city')][:l]
            fecha_meridiano =[]
            
            fecha = [i.text for i in driver.find_elements_by_tag_name('span.News-content-details-date')][:l]
            for i in fecha:
                i = i.split(' ')[0]
                variable = i.split('-')
                variable = variable[0][0:1]
                if variable in ('0','1','2','3','4','5','6','7','8','9'):
                    fecha_meridiano.append(dt_string)
                else:
                    a = dt_string.split('/')
                    variable_2 = i.split('-')
                    dia = variable_2[1]
                    mes = variable_2[0]
                    anio = a[2]
                    fecha_meridiano.append(MesNumero(dia + "-" +  mes + "-" + anio))
                    
            url = [i.get_attribute('href') for i in driver.find_elements_by_tag_name('a.no-hover, a.col-12')]
            variable_url = []
            variable_des = []
            for item in url:
                if item[0:4]=='http':
                    variable = item
                    variable_url.append(item)
                    driver.get(variable)
                else:
                    if item[:1] =='/':
                        variable = 'https://elmeridiano.co{urls}'.format(urls=item)
                        variable_url.append(variable)
                        driver.get(variable)
                    else:
                        variable = 'https://elmeridiano.co/{urls}'.format(urls=item)
                        variable_url.append(variable)
                        driver.get(variable)
                if len(titulo)!=0:
                    descripcion_larga = [i.text for i in driver.find_elements_by_tag_name('div.News-content-description, div.mb-3')]
                    descripcion_larga = [i.replace('\xa0',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\n',' ') for i in descripcion_larga]
                    descripcion_larga = [i.replace('\t',' ') for i in descripcion_larga]
                    descripcion_larga = ' '.join(descripcion_larga)
                    if descripcion_larga == '':
                        valor = 'Sin descripcion en la noticia'
                        variable_des.append(valor)
                    else:
                        variable_des.append(descripcion_larga)
                else:
                    variable_des.append('Error, pagina no encontrada')
            catego = []
            tipo = []
            for x in descripcion:
                vect_text1 = lemant(x)
                predi_categ = predCateg.predict([vect_text1])[0]
                predi_type = predType.predict([vect_text1])[0] 
                catego.append(predi_categ)
                tipo.append(predi_type)

            noticias = {} 
            noticias['title'] = titulo
            noticias['description']= descripcion
            noticias['description_long']= variable_des
            noticias['pubDate'] = fecha_meridiano
            noticias['category']= categoria
            noticias['link'] = variable_url
            noticias['medium']="El Meridiano"
            #--  Predict
            noticias['predi_type'] = tipo
            noticias['predi_categ'] = catego

            df_meridiano = pd.DataFrame(noticias,columns=['title', 'description', 'description_long', 'pubDate', 'link', 'category','medium','processDate','predi_type','predi_categ'])
            return df_meridiano

        else:
            df_meridiano = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
    
    except ValueError:
        df_meridiano = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})


fuente_sucre = fuente_meridiano('https://elmeridiano.co/sucre/judicial')
fuente_cordoba = fuente_meridiano('https://elmeridiano.co/cordoba/judicial')


if fuente_cordoba is None and fuente_sucre is None:
    df_meridiano = pd.DataFrame({'title': ['-'], 'description':['-'], 'description_long': ['-'], 'pubDate': ['-'],'link': ['-'], 'category': ['-'],'medium': ['-'], 'processDate': ['-'],'predi_type': ['FUERA_RANGO'], 'predi_categ': ['FUERA_RANGO']})
else:
    df_meridiano = pd.concat([fuente_cordoba, fuente_sucre])
    
driver.close()

## Union de Periodicos - Un solo DataFrame

In [ ]:
periodicos_union = [df_meridiano, df_nuevo_liberal, df_latarde, df_opinion, df_noticias_pilon, 
                    df_el_informador, df_la_razon, df_magdalena, df_colombiano, df_universal, df_blu, 
                    df_vanguardia, df_caracol, df_rcn_radio, df_noticias_PrensaLibre, df_hsb_noticias, 
                    df_nuevodia, df_noticias_ElTiempo, df_elpais, df_diarionorte, df_la_nacion, 
                    df_noticias_huila, df_siglo, df_diario_del_sur, df_putumayo, df_cronica_quindio, 
                    df_extra_judicial, df_cinaruco, df_la_patria, df_el_heraldo, df_policia, df_unidadvictimas, 
                    df_fuerza, df_ejercito, df_cauca, df_armada, df_el_espectador]
periodicos_union_df = pd.concat(periodicos_union)
periodicos_union_df = periodicos_union_df.assign(processDate=dt_string)
periodicos_union_df = periodicos_union_df.reset_index(drop=True)

In [ ]:
df_periodicos_limpios = periodicos_union_df[(periodicos_union_df['predi_type'] != 'FUERA_RANGO') | (periodicos_union_df['predi_categ'] != 'FUERA_RANGO')]
df_periodicos_limpios = df_periodicos_limpios.drop_duplicates(['title'], keep='last')
if dt_string[0] == '0':
    df_periodicos_limpios = df_periodicos_limpios[(df_periodicos_limpios['pubDate'] == dt_string) | (df_periodicos_limpios['pubDate'] == ayer) | (df_periodicos_limpios['pubDate'] == dt_string_adicional) | (df_periodicos_limpios['pubDate'] == ayer_adicional)]
else:
    df_periodicos_limpios = df_periodicos_limpios[(df_periodicos_limpios['pubDate'] == dt_string) | (df_periodicos_limpios['pubDate'] == ayer)]
df_periodicos_limpios = df_periodicos_limpios.reset_index(drop=True)

In [ ]:
df_periodicos_limpios.predi_type.value_counts()

In [ ]:
df_periodicos_limpios.predi_categ.value_counts()

In [ ]:
if len(error_page) > 0:
    f = open ('C:/Users/harold.patino/Documents/Periodicos/DeVops/Validar_periodico.txt','w')
    f.write(str(error_page))
    f.close()

In [ ]:
df_periodicos_limpios.to_csv('C:/Users/harold.patino/Documents/Periodicos/DeVops/Noticias_long.txt', sep='|', encoding='utf-8-sig')